<a href="https://colab.research.google.com/github/Rustam86/HSE-course-project/blob/main/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Настройка окружения





In [ ]:
!sudo apt-get install unrar

In [ ]:
!pip install biopython
!pip install tqdm
!pip install transformers
!pip install ete3

In [ ]:
# "Install" ZDNABERT
%cd /content
model = 'HG chipseq'
model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'

!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

In [ ]:
# "Install" ZHUNT
!wget https://raw.githubusercontent.com/Rustam86/HSE-course-project/main/zhunt3-alan.c
!gcc zhunt3-alan.c -lm -o zhunt3
!chmod +x /content/zhunt3

In [ ]:
# Install MAFFT
!apt-get install mafft

In [ ]:
!mafft --version

In [ ]:
# Install TrimAl
!git clone https://github.com/scapella/trimal.git
%cd trimal/source
!make

In [ ]:
!./trimal --version

In [ ]:
# Install IQ-TREE (takes some time)
%cd /content
!apt-get install -y libeigen3-dev
!git clone https://github.com/Cibiv/IQ-TREE.git
%cd IQ-TREE
!mkdir build
%cd build
!cmake ..
!make -j4

In [ ]:
!./iqtree --version

In [ ]:
import json
import logging
import os
import random
import sys
import subprocess
import tempfile
import time
from typing import Dict, List, Tuple, Callable, Optional, Union
import warnings

from ete3 import ClusterTree
import colorsys
from google.colab import files
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import scipy
import seaborn as sns
import torch
import Bio
from Bio import Entrez, SeqIO
from Bio.SeqUtils import gc_fraction, GC_skew, MeltingTemp as mt
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, fcluster, linkage, is_valid_linkage
from scipy import ndimage, stats
import sklearn
from sklearn.linear_model import LinearRegression
import torch
from torch import nn
import tqdm as tq
from tqdm import tqdm
import transformers
from transformers import BertForTokenClassification, BertTokenizer

In [ ]:
# Image resolution set to 300 dpi
plt.rcParams["figure.dpi"] = 300

In [ ]:
# Printing versions
print(' '.join(f"Python version: {sys.version}".split()[:3]))
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scipy version: {scipy.__version__}")
print(f"Sklearn version: {sklearn.__version__}")
print(f"Torch version: {torch.__version__}")
print(f"Seaborn version: {sns.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Biopython version: {Bio.__version__}")
print(f"TQDM version: {tq.__version__}")
print(f"Transformers version: {transformers.__version__}")

## Предварительно подготовленные данные

In [ ]:
# Get file with Coronaviridae taxonomy information
!wget https://raw.githubusercontent.com/Rustam86/HSE-course-project/main/Coronaviridae_taxonomic_info.csv
# Read the taxonomic data
taxa_db = pd.read_csv("/content/Coronaviridae_taxonomic_info.csv", encoding='ISO-8859-1', delimiter=';')

In [ ]:
# Dowload cornaviridae strains genbank files
!wget https://github.com/Rustam86/HSE-course-project/raw/main/species_gb_files.rar
!mkdir species_gb_files
!unrar x /content/species_gb_files.rar

In [ ]:
# Download json prediction files from github
!wget https://github.com/Rustam86/HSE-course-project/raw/main/Coronaviridae_strains_json.rar
!unrar x /content/Coronaviridae_strains_json.rar

In [ ]:
# Download clusterd dataframe csv files from github
!wget https://github.com/Rustam86/HSE-course-project/raw/main/clustered_data_frames_dnabert.rar
!unrar x clustered_data_frames_dnabert.rar

!wget https://github.com/Rustam86/HSE-course-project/raw/main/clustered_data_frames_zhunt.rar
!unrar x clustered_data_frames_zhunt.rar

In [ ]:
# Download row linkage matrices files from github
!wget https://github.com/Rustam86/HSE-course-project/raw/main/row_linkage_matrices_dnabert.rar
!unrar x row_linkage_matrices_dnabert.rar

!wget https://github.com/Rustam86/HSE-course-project/raw/main/row_linkage_matrices_zhunt.rar
!unrar x row_linkage_matrices_zhunt.rar

In [ ]:
# Download SARS-CoV-2 10k data from github
!wget https://github.com/Rustam86/HSE-course-project/raw/main/SARS-CoV-2-parts.rar
!unrar x SARS-CoV-2-parts.rar

In [ ]:
!wget https://raw.githubusercontent.com/Rustam86/HSE-course-project/main/meta_data_all_species.csv

In [ ]:
!wget https://raw.githubusercontent.com/Rustam86/HSE-course-project/main/sars2_meta_10k.csv

In [ ]:
!wget https://github.com/Rustam86/HSE-course-project/raw/main/SARS-CoV-2-full-genomes-only.rar
!unrar x SARS-CoV-2-full-genomes-only.rar

In [ ]:
!rm -r *.rar

## Переменные окружения

In [ ]:
# Set email (required for accessing NCBI databases via Entrez)
Entrez.email = "rustam@heydarov.ru"

ZDNABERT_PARAMS = {
    'model_confidence_threshold': 0.2,
    'minimum_sequence_length': 10,
    'tokenizer': BertTokenizer.from_pretrained('/content/6-new-12w-0/'),
    'model': BertForTokenClassification.from_pretrained('/content/6-new-12w-0/')
}

ZHUNT_PARAMS = {
    'zhunt_path': '/content/zhunt3',
    'score': 500,
    'window_size': 6,
    'min_size': 3,
    'max_size': 6
}

colors = ['#003f5c', '#ffb5a7', '#9c88ff', '#ff7b00', '#955196', '#b5838d', '#edc7b7', '#6b705c',
          '#f4d03f', '#d4af37', '#da627d', '#ff6e54', '#d1ccc0', '#7c7c7c', '#b7c0c7', '#c4aead',
          '#8d99ae', '#444e86', '#9d8189', '#6497b1', '#d6e2e9', '#e5989b', '#006d77', '#95afc0',
          '#dd5182', '#ffa600', '#5a189a', '#2e3440',  '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
          '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#7c1e30', '#b29e7c',
          '#2d2d87', '#beaed4', '#fdc086', '#bf5b17', '#f0027f', '#fbb4ae', '#fed9a6', '#b3cde3']


PREVALENT_HOSTS = {
    'Alphacoronavirus_1': 'Cats',
    'Alphacoronavirus_AMALF': 'Bats',
    'Alphacoronavirus_BT020': 'Bats',
    'Alphacoronavirus_CHB25': 'Bats',
    'Alphacoronavirus_HKU33': 'Bats',
    'Alphacoronavirus_WA1087': 'Bats',
    'Alphacoronavirus_WA2028': 'Bats',
    'Alphacoronavirus_WA3607': 'Bats',
    'Alphapironavirus_bona': 'Salmon',
    'Avian_coronavirus': 'Chicken',
    'Avian_coronavirus_9203': 'Chicken',
    'Bat_Hp-betacoronavirus_Zhejiang2013': 'Bats',
    'Bat_coronavirus_CDPHE15': 'Bats',
    'Bat_coronavirus_HKU10': 'Bats',
    'Beluga_whale_coronavirus_SW1': 'Whale',
    'Betacoronavirus_1': 'Human/Swine/Dog',
    'Bulbul_coronavirus_HKU11': 'Birds',
    'China_Rattus_coronavirus_HKU24': 'Rat',
    'Common_moorhen_coronavirus_HKU21': 'Birds',
    'Coronavirus_HKU15': 'Swine',
    'Duck_coronavirus_2714': 'Birds',
    'Eidolon_bat_coronavirus_C704': 'Bats',
    'Goose_coronavirus_CB17': 'Birds',
    'Hedgehog_coronavirus_1': 'Hedgehog',
    'Human_coronavirus_229E': 'Human',
    'Human_coronavirus_HKU1': 'Human',
    'Human_coronavirus_NL63': 'Human',
    'Human_coronavirus_OC43': 'Human',
    'Lucheng_Rn_rat_coronavirus': 'Rat',
    'Middle_East_respiratory_syndrome-related_coronavirus': 'Human',
    'Miniopterus_bat_coronavirus_1': 'Bats',
    'Miniopterus_bat_coronavirus_HKU8': 'Bats',
    'Mink_coronavirus_1': 'Mink',
    'Munia_coronavirus_HKU13': 'Birds',
    'Murine_coronavirus': 'Mice',
    'Myodes_coronavirus_2JL14': 'Myodes',
    'Myotis_ricketti_alphacoronavirus_Sax-2011': 'Bats',
    'NL63-related_bat_coronavirus_strain_BtKYNL63-9b': 'Bats',
    'Night_heron_coronavirus_HKU19': 'Birds',
    'Nyctalus_velutinus_alphacoronavirus_SC-2013': 'Bats',
    'Pipistrellus_bat_coronavirus_HKU5': 'Bats',
    'Pipistrellus_kuhlii_coronavirus_3398': 'Bats',
    'Porcine_epidemic_diarrhea_virus': 'Swine',
    'Rhinolophus_bat_coronavirus_HKU2': 'Bats',
    'Rhinolophus_ferrumequinum_alphacoronavirus_HuB-2013': 'Bats',
    'Rousettus_bat_coronavirus_GCCDC1': 'Bats',
    'Rousettus_bat_coronavirus_HKU9': 'Bats',
    'Scotophilus_bat_coronavirus_512': 'Bats',
    'Severe_acute_respiratory_syndrome_related_coronavirus': 'Human',
    'Severe_acute_respiratory_syndrome_related_coronavirus_2': 'Human',
    'Sorex_araneus_coronavirus_T14': 'Shrew',
    'Suncus_murinus_coronavirus_X74': 'Shrew',
    'Tylonycteris_bat_coronavirus_HKU4': 'Bats',
    'White-eye_coronavirus_HKU16': 'Birds',
    'Wigeon_coronavirus_HKU20': 'Birds'
}

## Вспомогательные функции

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """
    Custom JSON encoder that converts numpy types to standard Python types.

    Extends the standard JSON encoder class to handle numpy integer, floating-point,
    and array types, converting them to standard Python types.
    """

    def default(self, obj):
        """
        Override the default method to handle numpy types.

        Parameters:
        obj: Object to encode.

        Returns:
        Encoded object.
        """
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

def sum_intervals(intervals: List[Tuple[int, int]]) -> int:
    """
    Calculate the total sum of the lengths of given intervals, accounting for overlaps.

    Parameters:
    intervals (List[Tuple[int, int]]): List of intervals represented as tuples of start and end.

    Returns:
    int: Total length of all intervals, adjusted for overlaps. Returns 0 if the input list is empty.
    """
    if not intervals:
        return 0

    # Sort intervals based on start times
    intervals.sort(key=lambda x: x[0])

    merged = [intervals[0]]
    for current in intervals:
        last = merged[-1]
        if current[0] <= last[1]:
            merged[-1] = (last[0], max(last[1], current[1]))
        else:
            merged.append(current)

    return sum([end - start for start, end in merged])

def save_json(data: dict, filename: str) -> None:
    """
    Save data to a JSON file.

    Parameters:
    - data (dict): The Python dictionary to save.
    - filename (str): The name of the file where the data should be saved.

    Returns:
    None
    """
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4, cls=NumpyEncoder)

def read_json_file(file_path: str) -> Optional[dict]:
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except PermissionError:
        print(f"Permission denied for file: {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return None

def list_files(directory: str) -> List[str]:
    try:
        return [os.path.join(directory, filename)
                for filename in os.listdir(directory)
                if os.path.isfile(os.path.join(directory, filename))]
    except PermissionError:
        print(f"Permission denied for directory: {directory}")
        return []


def datetime_to_ordinal(dt):
    if pd.notna(dt):  # check if the value is not NaT
        return dt.toordinal()
    else:
        return None

def filter_arrays(arrays):
    """
    Filters arrays by removing those that contain NaN values or have a length of zero.

    Parameters:
    - arrays (list): A list of arrays.

    Returns:
    - filtered_arrays (list): A list of filtered arrays.
    """
    filtered_arrays = []
    for array in arrays:
        if np.isnan(array).any() or len(array) == 0:
            continue
        filtered_arrays.append(array)
    return filtered_arrays

def get_high_contrast_colors(n):
    HSV_tuples = [(x*1.0/n, 0.8, 0.9) for x in range(n)]
    RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)
    hex_colors = [f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}' for r,g,b in RGB_tuples]
    return hex_colors

def generate_high_contrast_colors(n):
    colors = []
    for i in range(n):
        # Cycle through hue
        hue = float(i) / n
        # Alternate between full and half brightness
        lightness = 0.5 if i % 2 == 0 else 0.8
        # Keep saturation constant
        saturation = 0.9
        # Convert to RGB
        r, g, b = [int(x * 255) for x in colorsys.hls_to_rgb(hue, lightness, saturation)]
        colors.append(f'#{r:02x}{g:02x}{b:02x}')
    return colors

def get_high_contrast_colors_gr(n):
    phi = 0.618033988749895
    h = random.random()  # random start value
    colors = []
    for _ in range(n):
        h += phi
        h %= 1
        colors.append(mcolors.hsv_to_rgb([h, 1, 1]))
    return colors

unique_values = [...]  # Your list of unique values
colors = get_high_contrast_colors(len(unique_values))
color_map = dict(zip(unique_values, colors))



## Основные функции

In [ ]:
def fetch_gb_files(accession_list: List[str], email: str, output_dir: str = ".") -> None:
    """
    Fetch GB (GenBank) files for a list of accession numbers.

    Parameters:
    accession_list (List[str]): List of accession numbers.
    email (str): Your email address. NCBI requires this to track usage.
    output_dir (str): Directory to save the GB files. Default is the current directory.

    Returns:
    None. GB files are saved to the specified directory.
    """

    # Set email for Entrez
    Entrez.email = email

    # Iterate through the list of accession numbers
    for accession in accession_list:
        # Fetch the data for the given accession number
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")

        # Form the filename where the data will be saved
        filename = f"{output_dir}/{accession}.gb"

        # Write the data to the file
        with open(filename, "w") as f:
            f.write(handle.read())

        # Close the handle
        handle.close()

        print(f"{accession}.gb saved to {output_dir}")

def list_proteins(gb_file: str) -> List[str]:
    """
    List all protein names from a GB (GenBank) file.

    Parameters:
    gb_file (str): Path to the GenBank file.

    Returns:
    List[str]: List of protein names.
    """

    protein_names = []

    # Parse the GenBank file
    for record in SeqIO.parse(gb_file, "genbank"):
        for feature in record.features:
            # Check if the feature is a coding sequence (CDS) and has a product
            if feature.type == "CDS" and "product" in feature.qualifiers:
                protein_names.append(feature.qualifiers["product"][0])

    return protein_names

def extract_protein_sequence(gb_file: str, protein_name: str) -> Optional[str]:
    """
    Extract amino acid sequence from a GB (GenBank) file given the name of a protein.

    Parameters:
    gb_file (str): Path to the GenBank file.
    protein_name (str): Name of the protein to extract the sequence for.

    Returns:
    str, None: Amino acid sequence as a string if found, else None.
    """

    # Parse the GenBank file
    for record in SeqIO.parse(gb_file, "genbank"):
        for feature in record.features:
            if feature.type == "CDS":
                # Check if the protein product name matches the given name
                if "product" in feature.qualifiers and feature.qualifiers["product"][0] == protein_name:
                    # Extract the protein sequence
                    return feature.qualifiers["translation"][0]

    return None

def write_to_fasta(seq_names: List[str], sequences: List[str], filename: str) -> None:
    """
    Write sequences to a FASTA file.

    Parameters:
    seq_names (List[str]): List of sequence names.
    sequences (List[str]): List of corresponding sequences.
    filename (str): Name of the output FASTA file.

    Returns:
    None. Writes the sequences to the specified FASTA file.
    """

    with open(filename, 'w') as f:
        for name, seq in zip(seq_names, sequences):
            # Write the name and sequence to the file in FASTA format
            f.write(f">{name}\n{seq}\n")

# Generic function to run sequence processing functions on a batch of sequences
def process_fasta_with_function(fasta_path: str, function: Callable, params: Dict) -> Dict[str, List[Tuple[int, int]]]:
    """
    Run a function on a multi-fasta file.

    Parameters:
    fasta_path (str): The path to the fasta file.
    function (Callable): The function to run on each sequence.
    params (Dict): A dictionary containing the parameters for the function.

    Returns:
    Dict[str, List[Tuple[int, int]]]: A dictionary where the keys are the sequence
                                      identifiers and the values are the results of
                                      the function for each sequence.
    """
    results = {}

    # Load all the sequences into a list to compute the total number of sequences
    sequences = list(SeqIO.parse(fasta_path, "fasta"))

    # Wrap the sequences list with tqdm for progress bar
    with tqdm(sequences, desc="Processing", unit="sequence") as pbar:
        for record in pbar:
            sequence_id = record.id
            sequence = str(record.seq)
            results[sequence_id] = function(sequence, **params)

    return results

# Function to predict Z-RNA regions with ZDNABERT
def run_zdnabert(seq_string: str, model:
                 torch.nn.Module, tokenizer: torch.nn.Module,
                 model_confidence_threshold: float = 0.2,
                 minimum_sequence_length: int = 10) -> List[Tuple[int, int]]:
    """
    Process a DNA or RNA sequence string using a given model and identify segments of significance.

    Parameters:
    - seq_string (str): The DNA or RNA sequence to be processed.
    - model (torch.nn.Module): Pretrained model for sequence prediction.
    - tokenizer (torch.nn.Module): Tokenizer to convert the sequence into model-compatible tokens.
    - model_confidence_threshold (float, optional): Threshold for considering a segment significant. Defaults to 0.2.
    - minimum_sequence_length (int, optional): Minimum length of a segment to consider. Defaults to 10.

    Returns:
    List[Tuple[int, int]]: A list of tuples representing the start and end positions
    of the predicted Z-DNA forming regions.
    """

    model.cuda()
    # Convert the sequence to k-mers
    k = 6
    kmer_seq = [seq_string.upper()[x:x+k] for x in range(len(seq_string)+1-k)]

    # Split the kmer sequence into pieces of a specified length with some overlap (padding)
    length, pad = 512, 16
    seq_pieces = [kmer_seq[st:min(st+512, len(kmer_seq))] for st in range(0, len(kmer_seq), length-pad)]

    # Use the model to predict on each sequence piece
    preds = []
    with torch.no_grad():
        for seq_piece in seq_pieces:
            input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
            outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1], axis=-1)[0, :, 1]
            preds.append(outputs.cpu().numpy())

    # Stitch together the predictions for each piece of the sequence
    res = np.array([])
    for seq in preds:
        res = np.concatenate([res[:-pad], seq])
    stitched_seq = res

    # Identify segments with prediction confidence above the threshold
    out = []
    labeled, max_label = scipy.ndimage.label(stitched_seq > model_confidence_threshold)
    for label in range(1, max_label+1):
        candidate = np.where(labeled == label)[0]
        candidate_length = candidate.shape[0]
        # Consider segments only if they are longer than the specified minimum sequence length
        if candidate_length > minimum_sequence_length:
            out.append((candidate[0], candidate[-1]))

    return out

# Function to predict Z-RNA regions with ZHUNT
def run_zhunt(seq_string: str, zhunt_path: str, score: int = 500,
              window_size: int = 6, min_size: int = 3,
              max_size: int = 6) -> List[Tuple[int, int]]:
    """
    Run the ZHunt program to predict Z-DNA forming regions in a DNA sequence.

    Parameters:
    - seq_string (str): The DNA or RNA sequence to be processed.
    - zhunt_path (str): The path to the ZHunt executable.
    - window_size (int): The window size for the ZHunt program. Default is 6.
    - min_size (int): The minimum size for the ZHunt program. Default is 3.
    - max_size (int): The maximum size for the ZHunt program. Default is 6.

    Returns:
    List[Tuple[int, int]]: A list of tuples representing the start and end positions
    of the predicted Z-DNA forming regions.
    """
    # Ensure the sequence only contains valid DNA bases
    # assert set(sequence).issubset({"A", "C", "G", "T", "N"}), "Invalid DNA sequence"

    # Create a temporary file
    file_descriptor, temp_file_path = tempfile.mkstemp()
    os.close(file_descriptor)

    try:
        # Write the sequence to the temporary file
        with open(temp_file_path, 'w') as temp_file:
            temp_file.write(seq_string)

        # Run the ZHunt program
        subprocess.run(
            [zhunt_path, str(window_size), str(min_size), str(max_size), temp_file_path],
            check=True, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL,
            input=seq_string, encoding='ascii'
        )

        # Read the ZHunt output into a DataFrame
        with open(temp_file_path + ".Z-SCORE", 'r') as zhunt_output:
            output_data = pd.read_csv(zhunt_output,
                             names=['Start', 'End', 'nu-1', 'nu-2', 'nu-3',
                                    'ZH-Score', 'Sequence', 'Conformation'],
                             skiprows=1, sep='\s+')

        # Filter the DataFrame to only include rows with a ZH-Score greater than 500
        filtered_data = output_data[output_data['ZH-Score'] > score]

        # Return a list of tuples representing the start and end positions of the predicted Z-DNA forming regions
        return list(zip(filtered_data['Start'], filtered_data['End']))

    except Exception as error:
        print(f"An error occurred while running ZHunt: {error}")

    finally:
        # Clean up the temporary files
        os.remove(temp_file_path)
        os.remove(temp_file_path + ".Z-SCORE")

def get_taxid(species_name: str) -> Optional[str]:
    """
    Retrieve the taxonomic identifier (taxid) for a given species name.

    Args:
        species_name (str): The name of the species for which the taxid is required.

    Returns:
        str: The taxid for the species, if found. None otherwise.
    """
    handle = Entrez.esearch(db="taxonomy", term=species_name)
    record = Entrez.read(handle)
    handle.close()
    try:
        return record["IdList"][0]
    except IndexError:
        return None

def fetch_genbank_taxid(taxid: str, filename: str, n: int) -> None:
    """
    Download a fixed number of random genomic GenBank files for a given taxid and save to a file.

    Args:
        taxid (str): The taxid for which GenBank files are required.
        filename (str): The name of the file to save the GenBank data.
        n (int): The fixed number of GenBank files to fetch.

    Returns:
        None
    """
    term = f'txid{taxid}[organism:exp] AND biomol_genomic[prop] AND ("25000"[SLEN] : "35000"[SLEN])'
    handle = Entrez.esearch(db="nuccore",
                            term=term,
                            retmax=10000000)
    record = Entrez.read(handle)
    id_list = record["IdList"]

    if not id_list:
        print(f"No GenBank records found for taxid {taxid}")
        return

    # Randomly select n IDs from id_list
    selected_ids = random.sample(id_list, min(n, len(id_list)))

    # Using tqdm to show progress bar
    with open(filename, "w") as out, tqdm(total=len(selected_ids), desc="Fetching GenBank records") as pbar:
        for gb_id in selected_ids:
            handle = Entrez.efetch(db="nuccore", id=gb_id, rettype="gb", retmode="text")
            out.write(handle.read())
            pbar.update(1)  # update progress bar for each GenBank record
            time.sleep(0.35)  # Optional: to avoid hitting API rate limits

def count_genbank_entries(taxid: str) -> int:
    """
    Count the number of GenBank entries for a given taxid.

    Args:
        taxid (str): The taxid for which the GenBank entry count is required.

    Returns:
        int: The count of GenBank entries associated with the taxid.
    """
    term = f'txid{taxid}[organism:exp] AND biomol_genomic[prop] AND ("25000"[SLEN] : "35000"[SLEN])'
    handle = Entrez.esearch(db="nuccore",
                            term=term,
                            retmax=100000000)
    record = Entrez.read(handle)
    return len(record["IdList"])

def newick_to_linkage(newick: str, label_order: list[str] = None) -> (np.ndarray, list[str]):
    """
    Convert a Newick formatted tree into a linkage matrix and retrieve corresponding labels.

    Parameters:
    - newick (str): A Newick formatted string representation of the tree.
    - label_order (list[str], optional): Desired order of labels in the output.
                                         If not provided, the order from the Newick string is used.

    Returns:
    - np.ndarray: Linkage matrix representation of the tree.
    - list[str]: List of labels corresponding to the tree nodes.

    Raises:
    - AssertionError: If there are labels in `label_order` which are not present in the Newick string.
    """

    # Convert Newick string to ClusterTree
    tree = ClusterTree(newick)

    # Get the cophenetic matrix and labels from the tree
    cophenetic_matrix, newick_labels = tree.cophenetic_matrix()
    cophenetic_matrix = pd.DataFrame(cophenetic_matrix, columns=newick_labels, index=newick_labels)

    # If a label order is provided, reorder the cophenetic matrix rows and columns accordingly
    if label_order is not None:
        # Identify missing and superfluous labels
        missing_labels = set(label_order).difference(set(newick_labels))
        superfluous_labels = set(newick_labels).difference(set(label_order))

        # Check for labels that are in `label_order` but not in Newick string
        assert len(missing_labels) == 0, f'Some labels are not in the newick string: {missing_labels}'

        # Warn if there are labels in the Newick string that are not used in `label_order`
        if len(superfluous_labels) > 0:
            logging.warning(f'Newick string contains unused labels: {superfluous_labels}')

        # Reorder the cophenetic matrix
        cophenetic_matrix = cophenetic_matrix.reindex(index=label_order, columns=label_order)

    # Convert the cophenetic matrix to a pairwise distance matrix
    pairwise_distances = squareform(cophenetic_matrix)

    # Return linkage matrix and labels
    return linkage(pairwise_distances), list(cophenetic_matrix.columns)

def total_interval_length(intervals_dict: Dict[str, List[Tuple[int, int]]]) -> Dict[str, int]:
    """
    Given a dictionary of intervals for each ID, this function calculates
    the total length of the intervals for each ID.

    Parameters:
    intervals_dict (Dict[str, List[Tuple[int, int]]]): Dictionary where key is ID and value is a list of intervals.

    Returns:
    Dict[str, int]: Dictionary where key is ID and value is total length of intervals.
    """

    length_dict = {}

    for key, intervals in intervals_dict.items():
        # Calculate total length for the current key by subtracting start from end for each interval
        total_length = sum([end - start for start, end in intervals])
        # Splitting the key and using the first part as the new key
        length_dict[key.split('.')[0]] = total_length

    return length_dict

def plot_tree_with_annotations(newick: str, numeric_values: dict,
                               taxa_db: pd.DataFrame, color_threshold: float = 0.9) -> None:
    """
    Plot a tree with annotations using a Newick formatted string along with numeric values
    represented as a bar plot beside the dendrogram.

    Parameters:
    - newick (str): A Newick formatted string representation of the tree.
    - numeric_values (dict): A dictionary containing the numeric values corresponding to the labels. e.g. {"label1": 5.6}
    - taxa_db (pd.DataFrame): A DataFrame containing 'Accession' and 'Prevalent host' data.
    - color_threshold (float, optional): The threshold for coloring branches. Defaults to 0.9.

    Returns:
    - None
    """

    # Convert newick string to linkage matrix and get labels
    linkage_matrix, labels = newick_to_linkage(newick)  # Ensure you've defined newick_to_linkage function

    switch = False

    if switch:
      labels = [dict(zip((taxa_db['Accession']), taxa_db['Species']))[id] for id in labels]

    # Create a triple subplot - one for the tree, one for numeric value bars, and one for the legend
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 6), gridspec_kw={'width_ratios': [2, 0.5, 0.5]})

    # Plot tree on the left subplot
    dendro_data = dendrogram(linkage_matrix, labels=labels, orientation='left', ax=ax1, color_threshold=color_threshold)

    # Generate host-related color mapping from the DataFrame
    host_groups = sorted(set(taxa_db['Prevalent host'].unique()))
    id_to_host = dict(zip(taxa_db['Accession'], taxa_db['Prevalent host']))
    palette = sns.color_palette("husl", len(host_groups))
    host_to_color = dict(zip(host_groups, palette))

    # Update the color for each label according to its prevalent host
    label_color_mapping = {}
    for label in dendro_data['ivl']:
        if switch:
            label = dict(zip((taxa_db['Species']), taxa_db['Accession']))[label]
        prevalent_host = id_to_host.get(label)
        label_color_mapping[label] = host_to_color.get(prevalent_host, "grey")


    # Plot tree on the left subplot
    dendro_data = dendrogram(linkage_matrix, labels=labels, orientation='left', ax=ax1, color_threshold=color_threshold)

    # Get the number of leaves (labels) in tree
    num_leaves = len(dendro_data['ivl'])

    # Get the maximum numeric value for x-limit of the middle subplot
    max_width = max(numeric_values.values())

    # Plot numeric values as colored boxes in the middle subplot
    for label, y in zip(dendro_data['ivl'], range(num_leaves)):
        if switch:
            label = dict(zip((taxa_db['Species']), taxa_db['Accession']))[label]
        box_width = numeric_values.get(label, 0)
        color = label_color_mapping.get(label, "grey")
        ax2.add_patch(plt.Rectangle((0, y*(ax1.get_ylim()[1]/num_leaves) + 0.25*ax1.get_ylim()[1]/num_leaves),
                                    box_width, 8, facecolor=color))

    # Set limits for the middle subplot and hide its y-axis
    ax2.set_xlim(0, max_width)
    ax2.set_ylim(ax1.get_ylim())
    ax2.yaxis.set_visible(False)

    # Hide x and y axes for the legend axis
    ax3.axis('off')

    # Populate the legend on the third axis
    for i, host in enumerate(host_groups):
        ax3.plot([], [], 'o', color=host_to_color[host], label=host)
    ax3.legend(title='Prevalent Host', loc='center')

    plt.tight_layout()
    plt.show()

def compute_jaccard_index(set_intervals1: List[Tuple[int, int]], set_intervals2: List[Tuple[int, int]]) -> float:
    """
    This function computes the Jaccard index between two sets of intervals.

    Parameters:
    - set_intervals1, set_intervals2 (list of tuples): The intervals to be compared.

    Returns:
    jaccard_index (float): The Jaccard index between the two sets of intervals.
    """

    # Compute the intersection of the two sets of intervals
    intersection_size = sum(min(interval2[1], interval1[1]) - max(interval2[0], interval1[0])
                            for interval1 in set_intervals1
                            for interval2 in set_intervals2
                            if interval1[1] > interval2[0] and interval2[1] > interval1[0])

    # Compute the size of intervals in set_intervals1 that do not overlap with set_intervals2
    non_overlap_size1 = sum(interval1[1] - interval1[0]
                            for interval1 in set_intervals1
                            if not any(interval1[1] > interval2[0] and interval2[1] > interval1[0]
                                       for interval2 in set_intervals2))

    # Compute the size of intervals in set_intervals2 that do not overlap with set_intervals1
    non_overlap_size2 = sum(interval2[1] - interval2[0]
                            for interval2 in set_intervals2
                            if not any(interval2[1] > interval1[0] and interval1[1] > interval2[0]
                                       for interval1 in set_intervals1))

    # Compute the size of the union of the two sets of intervals
    union_size = intersection_size + non_overlap_size1 + non_overlap_size2

    # Compute the Jaccard index
    if union_size:
        jaccard_index = intersection_size / union_size
    else:
        jaccard_index = 1

    return jaccard_index


def create_clustered_dataframe(zrna_intervals_dict: Dict) -> Tuple[pd.DataFrame, np.ndarray]:
    """
    This function creates a clustered dataframe based on Jaccard indices of intervals.

    Parameters:
    - zrna_intervals_dict (Dict): The dictionary containing Z-RNA interval data.

    Returns:
    clustered_dataframe (pd.DataFrame): The clustered dataframe.
    row_linkage_matrix (np.ndarray): The hierarchical clustering encoded as a linkage matrix.
    """

    # Parse the intervals file
    intervals_dict = zrna_intervals_dict

    sequence_ids = list(intervals_dict.keys())

    # Check if there are enough sequences
    if len(sequence_ids) < 2:
        return pd.DataFrame({list(zrna_intervals_dict.keys())[0]: 0},
                            index=list(zrna_intervals_dict.keys()),
                            columns=list(zrna_intervals_dict.keys())), None, None

    # Compute the Jaccard index matrix with a single progress bar
    print("Computing Jaccard index matrix...")
    intervals_list = list(intervals_dict.items())
    interval_count = len(intervals_list)
    sequence_labels = [intervals_list[i][0] for i in range(interval_count)]

    jaccard_index_matrix = np.zeros((interval_count, interval_count))
    with tqdm(total=interval_count**2, desc="Computing Jaccard indices") as pbar:
        for i in range(interval_count):
            for j in range(interval_count):
                jaccard_index_matrix[i][j] = compute_jaccard_index(intervals_list[i][1], intervals_list[j][1])
                pbar.update(1)

    jaccard_dataframe = pd.DataFrame(jaccard_index_matrix, index=sequence_labels, columns=sequence_labels)

    # Perform hierarchical clustering on rows and columns
    print("Performing hierarchical clustering on rows...")
    row_linkage_matrix = linkage(jaccard_dataframe.values, method='average', metric='euclidean')
    print("Performing hierarchical clustering on columns...")
    column_linkage_matrix = linkage(jaccard_dataframe.values.T, method='average', metric='euclidean')

    # Reorder the dataframe based on the clustering
    print("Reordering dataframe based on clustering...")
    row_dendrogram = dendrogram(row_linkage_matrix, no_plot=True)
    column_dendrogram = dendrogram(column_linkage_matrix, no_plot=True)
    clustered_dataframe = jaccard_dataframe.iloc[row_dendrogram['leaves'], column_dendrogram['leaves']]

    return clustered_dataframe, row_linkage_matrix

def extract_regions_from_genbank_id(genbank_id: str) -> List[Tuple[str, int, int]]:
    """
    Extract genomic regions including CDS (excluding "ORF1ab polyprotein" and "ORF1a polyprotein"),
    mat_peptides, and UTRs from a given GenBank ID.

    Parameters:
    - genbank_id (str): The GenBank ID to fetch.

    Returns:
    - List[Tuple[str, int, int]]: A list of tuples where each tuple contains:
      * The product name or region type (str).
      * The start coordinate (int).
      * The end coordinate (int).
    """


    # Fetch the GenBank record
    handle = Entrez.efetch(db="nucleotide", id=genbank_id, rettype="gb", retmode="text")

    regions = []

    for record in SeqIO.parse(handle, "genbank"):
        for feature in record.features:

            # Extract CDS coordinates and product names
            if feature.type == "CDS":
                product = feature.qualifiers.get('product', ['unknown'])[0]
                if product not in ["ORF1ab polyprotein", "ORF1a polyprotein"]:
                    start = int(feature.location.start) + 1
                    end = int(feature.location.end)
                    regions.append((product, start, end))

            # Extract mat_peptide coordinates and product names
            elif feature.type == "mat_peptide":
                start = int(feature.location.start) + 1
                end = int(feature.location.end)
                product = feature.qualifiers.get('product', ['unknown'])[0]
                regions.append((product, start, end))

            # Extract UTR coordinates
            elif feature.type == "5'UTR" or feature.type == "3'UTR":
                start = int(feature.location.start) + 1
                end = int(feature.location.end)
                regions.append((feature.type, start, end))

    handle.close()

    # Remove duplicates and sort regions based on start coordinate
    regions = list(set(regions))
    regions.sort(key=lambda x: x[1])

    return regions

def plot_zna_regions(virus_name: str, sorted_keys: List[str],
                     regions_1: Dict[str, List[Tuple[int, int]]],
                     regions_2: Dict[str, List[Tuple[int, int]]],
                     genomic_regions_data: List[Tuple[str, int, int]],
                     colors: List[str],
                     figsize: Tuple[int, int] = (30, 15),
                     save: bool = False) -> None:
    """
    Plot Z-DNA regions predicted by two different methods, Z-HUNT and Z-DNABERT,
    across different strains of a virus species.

    Args:
    - virus_name (str): The name of the virus.
    - sorted_keys (list): A list of keys (strains names) used to order the data.
    - regions_1 (dict): A dictionary containing the Z-DNA regions predicted by Z-HUNT.
                      Each key is a strain name and each value is a list of start
                      and end positions of the Z-DNA regions.
    - regions_2 (dict): A dictionary containing the Z-DNA regions predicted by Z-DNABERT.
                      Structured the same as regions_1.
    - genomic_regions_data (list): A list of tuples where each tuple contains the name of a gene and its
                 start and end positions.
    - colors (list): A list of colors to use for the horizontal lines representing genes.
    - figsize (tuple, optional): The size of the figure to plot. Defaults to (30, 15).
    - save (bool, optional): If True, save the plot in PNG and PDF formats. Defaults to False.

    Returns:
    None. Shows the plot inline or saves it to file.
    """

    fig, ax = plt.subplots(figsize=figsize)
    genes, starts, ends = zip(*genomic_regions_data)

    for i in range(len(genomic_regions_data)):
        ax.hlines(y=0, xmin=starts[i], xmax=ends[i], linewidth=10, color=colors[i])

    lim = max(ends) + 50
    ax.set_xlim(0, lim)

    ax.yaxis.set_visible(False)
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['bottom'].set_position(('outward', 10))

    xticks = []
    xlabels = []
    for i in range(len(genomic_regions_data)):
        xticks.append((starts[i] + ends[i]) / 2)
        xlabels.append(genomic_regions_data[i][0])
    ax.set_xticks(xticks)
    ax.set_xticklabels(xlabels, fontsize=10)

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    patches = [plt.Rectangle((0, 0), 1, 1, color=colors[i]) for i in range(len(genomic_regions_data))]
    ax.legend(patches, genes, loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=5, fontsize=10)

    plt.subplots_adjust(bottom=0.35)

    counter = 1
    for key in sorted_keys:
        if key in regions_1.keys():
            adata = [(regions_1[key][1:][i][0], regions_1[key][1:][i][1]) for i in range(len(regions_1[key][1:]))]
            if adata:
              starts, ends = zip(*adata)
              for i in range(len(adata)):
                  ax.hlines(y=counter, xmin=starts[i], xmax=ends[i], linewidth=5, colors='#e8071a', label='Z-HUNT')
              counter += 0.25

    counter = 1
    for key in sorted_keys:
        if key in regions_2.keys():
            adata = [(regions_2[key][1:][i][0], regions_2[key][1:][i][1]) for i in range(len(regions_2[key][1:]))]
            if adata:
              starts, ends = zip(*adata)
              for i in range(len(adata)):
                  ax.hlines(y=counter, xmin=starts[i], xmax=ends[i], linewidth=5, colors='#07a1e8', label='Z-DNABERT')
              counter += 0.25

    n_strains = len(regions_1)
    title = f"{virus_name}\n\n(number of strains: {n_strains})"
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Position (nucleotides)', fontsize=12)

    if save:
        fig.savefig(f"{virus_name}.png", bbox_inches='tight')
        fig.savefig(f"{virus_name}.pdf", bbox_inches='tight')
    else:
        plt.show()

def extract_genbank_data(file_list: list) -> pd.DataFrame:
    """
    Extracts various data and calculations from a list of GenBank files.

    Parameters:
    - file_list (list): A list of paths to GenBank files.

    Returns:
    - pd.DataFrame: A DataFrame containing the extracted details for each GenBank file.
    """

    data = []

    for file_path in file_list:
        with open(file_path, "r") as handle:
            for record in SeqIO.parse(handle, "genbank"):
                # General Details
                accession_with_version = record.id
                virus_species_from_file = file_path.split("/")[-1].split(".")[0]

                # Taxonomy Details
                taxonomy = record.annotations.get('taxonomy', [])
                subfamily = taxonomy[8] if len(taxonomy) >= 9 else ""
                genus = taxonomy[9] if len(taxonomy) >= 10 else ""
                subgenus = taxonomy[10] if len(taxonomy) >= 11 else ""
                species = taxonomy[11] if len(taxonomy) >= 12 else ""

                # Source Features
                host = None
                collection_date = None
                for feature in record.features:
                    if feature.type == "source":
                        host = feature.qualifiers.get("host", [None])[0]
                        collection_date = feature.qualifiers.get("collection_date", [None])[0]
                        break
                if not collection_date:
                    collection_date = record.annotations.get('date', '')

                # Sequence Calculations
                seq_length = len(record.seq)
                gc_content = gc_fraction(record.seq) * 100
                melting_temperature = mt.Tm_NN(record.seq)

                gc_skew_values = GC_skew(record.seq, window=100)
                gc_skew_avg = sum(gc_skew_values) / len(gc_skew_values) if gc_skew_values else 0

                data.append([
                    virus_species_from_file, accession_with_version, subfamily,
                    genus, subgenus, species, host, seq_length, gc_content,
                    melting_temperature, gc_skew_avg,
                    collection_date
                ])

    columns = [
        'virus_species_from_file', 'accession_with_version', 'subfamily',
        'genus', 'subgenus', 'species', 'host', 'Sequence length', 'GC content',
        'Melting temperature', 'GC skew', 'date'
    ]
    df = pd.DataFrame(data, columns=columns)

    df['datetime'] = pd.to_datetime(df['date'], errors='coerce')

    return df

def plot_heatmap_with_dendrogram(clustered_dataframe: pd.DataFrame,
                                 row_linkage_matrix: np.ndarray,
                                 color_threshold: float,
                                 title: str = 'Heat map with dendrogram',
                                 figsize: Tuple[int, int] = (10, 10),
                                 save_figure: bool = False,
                                 file_name: str = 'results',
                                 close_figure: bool = False) -> None:

    # Validate input data
    if not isinstance(clustered_dataframe, pd.DataFrame) or clustered_dataframe.ndim != 2:
        raise ValueError("Input `clustered_dataframe` should be a 2D DataFrame.")

    if not is_valid_linkage(row_linkage_matrix):
        raise ValueError("Input `row_linkage_matrix` is not a valid linkage matrix.")

    if not np.issubdtype(clustered_dataframe.values.dtype, np.number):
        raise ValueError("Heatmap can only be plotted with numerical data.")

    # Create a figure to contain the plot elements
    figure = plt.figure(figsize=figsize)

    # Create a gridspec to handle the layout
    grid_spec = figure.add_gridspec(2, 2, width_ratios=[0.05, 1], height_ratios=[0.2, 1], wspace=0.02, hspace=0.02)

    # Add dendrogram on top
    dendrogram_axis = figure.add_subplot(grid_spec[0, 1])
    with plt.rc_context({'lines.linewidth': 0.5}):
        dendro = dendrogram(row_linkage_matrix, ax=dendrogram_axis, orientation='top', color_threshold=color_threshold)
    dendrogram_axis.axis('off')

    # Create a color map
    clusters = fcluster(row_linkage_matrix, color_threshold, criterion='distance')
    unique_clusters = len(np.unique(clusters))

    colors = sns.color_palette('tab20', n_colors=unique_clusters)
    color_map = dict(enumerate(colors, 1))

    # Change the color of each line to match the cluster colors
    for i, d, c in zip(dendro['icoord'], dendro['dcoord'], clusters):
        for j in range(4):
            x = 0.5 * sum(i[j:j+2])
            y = d[j]
            dendrogram_axis.plot(x, y, color=color_map[c])

    # Add heatmap
    heatmap_axis = figure.add_subplot(grid_spec[1, 1])
    sns.heatmap(clustered_dataframe, annot=False, ax=heatmap_axis, cbar=False, xticklabels=False, yticklabels=False)

    # Add title to the entire figure
    figure.suptitle(title, fontsize=10, y=0.91)

    plt.tick_params(labelsize=5)
    if save_figure:
        plt.savefig(f"{file_name}_heatmap.png")
        plt.savefig(f"{file_name}_heatmap.pdf")
    if close_figure:
        plt.close()
    plt.show()

def optimal_dendrogram_threshold_and_clusters(linkage_matrix: np.ndarray, N: int) -> float:
    """
    Find the threshold for the linkage matrix such that the number of clusters is
    approximately N. If a suitable threshold cannot be found, return a default value.
    """

    # Starting threshold
    threshold = linkage_matrix[-N, 2] if N < linkage_matrix.shape[0] else linkage_matrix[0, 2]

    # Small value to adjust the threshold in each step
    delta = 0.001

    # Default threshold as the average of all distances
    default_threshold = np.mean(linkage_matrix[:, 2])

    while True:
        clusters = fcluster(linkage_matrix, threshold, criterion='distance')
        unique_clusters = len(np.unique(clusters))

        if unique_clusters == N:
            return threshold

        if unique_clusters < N:
            threshold -= delta
        else:
            threshold += delta

        # Return default threshold if search goes out of bounds
        if threshold < 0 or threshold > linkage_matrix[-1, 2]:
            print("Warning: Couldn't find a suitable threshold for the specified number of clusters.")
            print(f"Using a default threshold of {default_threshold}")
            return default_threshold

def plot_time_length_regression(
    virus_name: str,
    meta_df: pd.DataFrame,
    intervals_column: str,
    color_threshold: float = None,
    clustered_dataframe: pd.DataFrame = None,
    row_linkage_matrix: np.ndarray = None,
    grouping_column: Optional[str] = None,
    title: str = 'Time and ZNA length regression',
    legend: str = 'Cluster: ',
    remove_outliers: Tuple[bool, int] = (False, 3),
    figsize: Tuple[int, int] = (10, 5),
    point_size: int = 15,
    save_figure: bool = False,
    normalize: bool = False,
    file_name: str = 'results',
    legend_loc: str = 'best',
    close_figure: bool = False) -> None:
    """
    Plot a time-length regression of Z-RNA regions' length against the collection date for different lineages.

    Parameters:
    - virus_name (str): Name of the virus to filter data.
    - meta_df (pd.DataFrame): DataFrame containing the data to be plotted.
    - intervals_column (str): The column name for length intervals in `meta_df`.
    - clustered_dataframe (pd.DataFrame): DataFrame with clustered data.
    - row_linkage_matrix (np.ndarray): Linkage matrix for hierarchical clustering.
    - color_threshold (float): Threshold to use for coloring clusters.
    - grouping_column (Optional[str]): The column for grouping data; defaults to None.
    - title (str): Title of the plot; defaults to 'Time and ZNA length regression'.
    - legend (str): Base string for the legend of each cluster; defaults to 'Cluster: '.
    - remove_outliers (Tuple[bool, int]): Indicator and z-score threshold for outlier removal; defaults to (False, 3).
    - figsize (Tuple[int, int]): Figure size; defaults to (10, 5).
    - point_size (int): The size of points in the scatter plot; defaults to 8.
    - save_figure (bool): Whether to save the figure as a PNG; defaults to False.
    - normalize (bool): Whether to normalize the data; defaults to False.
    - file_name (str): The name of the file to save the figure; defaults to 'results'.
    - legend_loc (str): The location of the legend; defaults to 'best'.
    - close_figure (bool): Whether to close the figure after plotting; defaults to False.

    Returns:
    None

    This function first processes the input data to ensure it's ready for plotting.
    It then uses this data to plot the time-length regression and optionally a regression line.
    The function can handle and visualize different lineage clusters,
    providing a comprehensive view of the data spread across time and length.
    Additional features include outlier removal, data normalization, and customized figure saving.
    """

    # Make an explicit copy
    df_copy = meta_df[meta_df['virus_species_from_file'] == virus_name].copy()

    # Convert 'date' to datetime format only once
    df_copy['date'] = pd.to_datetime(df_copy['date'], errors='coerce')

    # Normalize the data if specified
    if normalize:
        df_copy['Normalized_Length'] = df_copy[intervals_column] / df_copy['Sequence Length']
        length_column = 'Normalized_Length'
    else:
        length_column = intervals_column

    # Convert the collection date to datetime and ordinal
    df_copy['date'] = pd.to_datetime(df_copy['date'], errors='coerce')
    df_copy = df_copy.dropna(subset=['date'])
    df_copy['Date_Ordinal'] = df_copy['date'].apply(lambda x: x.toordinal())

    # Remove outliers if specified
    if remove_outliers[0]:
        z_scores = df_copy[['Date_Ordinal', intervals_column]].apply(
            lambda x: (x - x.mean()) / x.std()
        )
        df_copy = df_copy[
            (np.abs(z_scores['Date_Ordinal']) <= remove_outliers[1]) &
            (np.abs(z_scores[intervals_column]) <= remove_outliers[1])
        ]

    # Fit a linear regression model
    X = df_copy[['Date_Ordinal']]
    y = df_copy[length_column]
    regression_model = LinearRegression()
    regression_model.fit(X, y)
    y_pred = regression_model.predict(X)

    # Create the figure
    plt.figure(figsize=figsize)

    if grouping_column != None:
        unique_lineages = df_copy[grouping_column].unique()
        colors = mpl.rcParams['axes.prop_cycle'].by_key()['color'][1:len(unique_lineages)+2]
        color_map = dict(zip(sorted(unique_lineages), colors))
        # Plot the data points for each cluster
        for linage in set(df_copy[grouping_column]):
            plt.scatter(df_copy[df_copy[grouping_column] == linage]['date'],
                        df_copy[df_copy[grouping_column] == linage][length_column],
                        color=color_map[linage], label=f"{linage}", s=point_size)
    else:
        grouping_column = 'leaves_color'
        dendro = dendrogram(row_linkage_matrix, color_threshold=color_threshold, no_plot=True)
        d = dict(zip(clustered_dataframe.columns, dendro['leaves_color_list']))
        df_copy['leaves_color'] = df_copy['accession_with_version'].apply(lambda x: d.get(x))

        unique_lineages = df_copy[grouping_column].unique()
        colors = mpl.rcParams['axes.prop_cycle'].by_key()['color'][1:len(unique_lineages)+2]
        color_map = dict(zip(sorted(unique_lineages), colors))
        # Plot the data points for each cluster
        for linage in set(df_copy[grouping_column]):
            plt.scatter(df_copy[df_copy[grouping_column] == linage]['date'],
                        df_copy[df_copy[grouping_column] == linage][length_column],
                        color=color_map[linage], label=f"{linage}", s=point_size)

    # Plot the regression line
    plt.plot(df_copy['date'], y_pred, color='red', label='Regression Line')

    # Get the slope and intercept of the regression line
    slope = regression_model.coef_[0]
    intercept = regression_model.intercept_

    # Create a list of legend elements
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label=f"{legend} {linage}",
               markerfacecolor=color, markersize=10)
        for linage, color in color_map.items()
    ]
    legend_elements.append(Line2D([0], [0], color='red', lw=2, label='Regression Line'))
    legend_elements.append(Line2D([0], [0], marker='None', color='w',
                                  label=f"Slope = {slope:.2f}\nIntercept = {intercept:.2f}"))

    # Customize the plot
    plt.legend(handles=legend_elements, loc=legend_loc, framealpha=0.5)
    plt.grid(visible=True, which='major', axis='both', linestyle='-')
    plt.title(title)
    plt.xlabel('Collection Date')
    plt.ylabel('Z-RNA regions total length')

    # Save the figure if specified
    if save_figure:
        plt.savefig(f"{file_name}_heatmap.png")
        plt.savefig(f"{file_name}_heatmap.pdf")

    # Close the figure if specified
    if close_figure:
        plt.close()

    # Show the plot
    plt.show()

def remove_outliers(df: pd.DataFrame, column_names: list, multiplier: float = 3) -> pd.DataFrame:
    """
    Removes outliers from a DataFrame based on values in specified columns using the IQR method.

    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        column_names (list): A list of column names in which to check for outliers.
        multiplier (float): The multiplier for the IQR. Defaults to 1.5.
        Increase this value to be more lenient with outliers, and decrease it to be stricter.

    Returns:
        pd.DataFrame: DataFrame with outliers removed.
    """

    for column_name in column_names:
        # Calculate Q1, Q2, and IQR
        Q1 = df[column_name].quantile(0.25)
        Q3 = df[column_name].quantile(0.75)
        IQR = Q3 - Q1

        # Define bounds
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR

        # Filter the data frame
        df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    return df

def draw_boxplot_species(
        data_subset: pd.DataFrame,
        category_label: str = 'Virus name',
        value_label: str = 'ZDNABERT intervals total length',
        plot_title: str = '',
        figure_size: Tuple[int, int] = (15, 6),
        text_position: Tuple[float, float] = (0.02, 0.02),
        legend_location: str = 'best') -> None:
    """
    Draws a boxplot for certain categories.

    Parameters:
    data_subset (pd.DataFrame): The subset of the data to be plotted.
    category_label (str): The name of the column representing the category.
    value_label (str): The name of the column representing the values.
    plot_title (str): The title of the plot.
    figure_size (Tuple[int, int]): The size of the figure.
    text_position (Tuple[float, float]): The position of the text inside the plot.
    legend_location (str): The location of the legend.

    Returns:
    None
    """
    # The unique categories
    categories = list(data_subset[category_label].unique())

    # Mapping English letters to Greek letters
    greek_dict = {
        'alphacoronavirus': r'$\alpha$',
        'betacoronavirus': r'$\beta$',
        'gammacoronavirus': r'$\gamma$',
        'deltacoronavirus': r'$\delta$',
        'alphapironavirus': r'$\pi$',
    }

    # Specify the species groups
    species_groups = ['Bats', 'Birds', 'Cats', 'Chicken', 'Hedgehog', 'Human',
                      'Human/Swine/Dog', 'Mice', 'Mink', 'Myodes', 'Rat', 'Salmon',
                      'Shrew', 'Swine', 'Whale']

    # Filter the DataFrame to only include rows where 'species' is in species_groups
    data_subset = data_subset[data_subset['Host'].isin(species_groups)]

    # Sort dataframe by species
    data_subset = data_subset.sort_values(by=['Host', 'virus_species_from_file'])

    # Store the 'Intervals total length' for each category
    category_values = []
    for category in categories:
        values = data_subset[data_subset[category_label] == category][value_label].values
        if not np.isnan(values).any():
            category_values.append(values)

    # Assuming that this function exists
    category_values = filter_arrays(category_values)

    # Perform the ANOVA
    f_value, p_value = stats.f_oneway(*category_values)

    # Create a figure and axis for the plot
    fig, ax = plt.subplots(figsize=figure_size)

    # Create the boxplot with seaborn
    box_plot = sns.boxplot(
        x=category_label,
        y=value_label,
        hue='Host',
        data=data_subset,
        dodge=False
    )

    # Add a title and labels
    ax.set_title(plot_title)
    ax.set_xlabel('Viruses')
    ax.set_ylabel(f'Z-RNA {value_label}')

    # Get color palette
    palette = sns.color_palette("husl", 20)

    # Create a mapping of labels to species
    label_to_species = data_subset.groupby(category_label)['Host'].agg(pd.Series.mode).to_dict()

    # Create a mapping of species to colors
    species_to_colors = dict(zip(species_groups, palette))

    # Mapping taxa to colors
    taxa_to_colors = {
        'alphapironavirus': '#FF6B6B',  # Light Red
        'alphacoronavirus': '#4ECDC4',  # Turquoise
        'betacoronavirus': '#556270',   # Dark Grayish Blue
        'deltacoronavirus': '#C7F464',  # Light Yellow Green
        'gammacoronavirus': '#FFA577'   # Light Orange
    }

    # Apply the colors to the labels based on the most common species for that label
    for i, label_text in enumerate(ax.get_xticklabels()):
        species = label_to_species[label_text.get_text()]
        color = species_to_colors[species]
        label_text.set_color(color)

        # Extract the corresponding 'taxa' value for the label
        taxa = data_subset[data_subset[category_label] == label_text.get_text()]['genus'].unique()[0]
        # Convert first letter to Greek equivalent
        taxa = taxa.lower()
        taxa_greek = greek_dict.get(taxa, taxa[0])

        # Update y position to below y=0 and set color based on taxa
        ax.text(
            (i + 0.5) / len(categories), 0.01, taxa_greek,
            horizontalalignment='center', size='small', color='black',
            weight='semibold', transform=ax.transAxes,
            bbox=dict(facecolor=taxa_to_colors[taxa], alpha=0.4, boxstyle='round,pad=0.2')
        )

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=10, fontweight='bold')

    # Set the legend title
    ax.legend(title='Host species', loc=legend_location)

    # Add the F-value, p-value, and test name inside the plot
    anova_text = f'ANOVA Test\nF-value: {f_value:.2f}\nP-value: {p_value:.4f}'
    ax.text(
        *text_position, anova_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='bottom', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5)
    )

    # Show the plot
    plt.show()

def pango_to_who(pango_lineage: str) -> str:
    """
    Converts Pango lineage of SARS-CoV-2 virus to WHO label.

    Args:
        pango_lineage (str): A string representing the Pango lineage of the virus.

    Returns:
        str: A string representing the WHO label.
    """
    pango_lineage = pango_lineage.upper()

    pango_to_who_map = {
        "B.1.1.7": "Alpha",
        "Q": "Alpha",
        "B.1.351": "Beta",
        "P.1": "Gamma",
        "B.1.617.2": "Delta",
        "B.1.617.1": "Kappa",
        "B.1.617.3": "Kappa",
        "B.1.427": "Epsilon",
        "B.1.429": "Epsilon",
        "B.1.525": "Eta",
        "B.1.526": "Iota",
        "C.37": "Lambda",
        "B.1.621": "Mu",
        "B.1.621.1": "Mu",
        "BA.1": "Omicron",
        "BA.2": "Omicron",
        "BA.4": "Omicron",
        "BA.5": "Omicron",
        "BA.2.12.1": "Omicron",
        "BA.2.75": "Omicron",
        "BQ.1": "Omicron",
        "XBB.1.5": "Omicron",
        "XBB.1.16": "Omicron",
        "P.2": "Zeta"
    }

    for pango, who in pango_to_who_map.items():
        if pango_lineage.startswith(pango):
            return who

    return "Unknown"

def draw_boxplot_who(
        df_subset: pd.DataFrame,
        label: str = 'WHO',
        values: str = 'Intervals Total Length',
        title: str = '',
        figsize: Tuple[int, int] = (15, 6),
        pos: Tuple[float, float] = (0.02, 0.02)) -> None:
    """
    Draws a boxplot for a given DataFrame.

    Parameters:
    df_subset (pd.DataFrame): The subset of the data to be plotted.
    label (str): The name of the column representing the category.
    values (str): The name of the column representing the values.
    title (str): The title of the plot.
    figsize (Tuple[int, int]): The size of the figure.
    pos (Tuple[float, float]): The position of the text inside the plot.

    Returns:
    None
    """
    # Pangolin values
    pangolin_values = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon',
                       'Eta', 'Iota', 'Kappa', 'Lambda', 'Mu',
                       'Omicron', 'Unknown', 'Zeta']

    # Create an empty list to store the 'Intervals total length' for each 'Pangolin' value
    data = []

    # For each 'Pangolin' value, append the 'Intervals total length' to the data list
    for pangolin in pangolin_values:
        data.append(df_subset[df_subset[label] == pangolin][values].values)

    # Assuming that this function exists
    data = filter_arrays(data)

    # Perform the ANOVA
    f_value, p_value = stats.f_oneway(*data)

    # Create a figure and axis for the plot
    fig, ax = plt.subplots(figsize=figsize)

    # Create the boxplot with seaborn
    sns.boxplot(x=label, y=values, data=df_subset, order=pangolin_values, ax=ax)

    # Add a title and labels
    ax.set_title(title)
    ax.set_xlabel(label)
    ax.set_ylabel(f'Z-RNA {values}')

    # Add the F-value, p-value, and test name inside the plot
    anova_text = f'ANOVA Test\nF-value: {f_value:.2f}\nP-value: {p_value:.2f}'
    ax.text(
        *pos, anova_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='bottom', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5)
    )

    # Show the plot
    plt.show()

def plot_time_length_linages_boxplot(
    meta_df: pd.DataFrame,
    length_column: str = '',
    title: str = '',
    remove_outliers: Tuple[bool, int] = (False, 3),
    figsize: Tuple[int, int] = (14, 8),
    text_position: Tuple[float, float] = (0.01, 0.90),
    save_figure: bool = False,
    normalize: bool = False,
    file_name: str = 'results',
    legend_loc: str = 'best',
    close_figure: bool = False
) -> None:

    # Copy the dataframe
    df_copy = meta_df.copy()

    # Normalize the data if specified
    if normalize:
        df_copy['Normalized_Length'] = df_copy['Intervals Total Length'] / df_copy['Sequence Length']
        length_column = 'Normalized_Length'

    # Convert the collection date to datetime and extract year and month
    df_copy['datetime'] = pd.to_datetime(df_copy['datetime'], errors='coerce')
    df_copy['Year_Month'] = df_copy['datetime'].dt.to_period('M')

    # Order by year and month
    df_copy['Year_Month'] = df_copy['Year_Month'].astype(
        CategoricalDtype(
            categories=sorted(df_copy['Year_Month'].unique()),
            ordered=True
        )
    )

    # Remove outliers if specified
    if remove_outliers[0]:
        z_scores = df_copy[[length_column]].apply(lambda x: (x - x.mean()) / x.std())
        df_copy = df_copy[(np.abs(z_scores[length_column]) <= remove_outliers[1])]

    # Create the figure
    fig, ax = plt.subplots(figsize=figsize)

    # Plot the boxplot for each month using seaborn
    sns.boxplot(x="Year_Month", y=length_column, data=df_copy, palette='viridis', width=0.5, ax=ax)

    # Perform ANOVA
    groups = [group[length_column].dropna() for name, group in df_copy.groupby('Year_Month')]
    f_value, p_value = stats.f_oneway(*groups)

    # Add the F-value, p-value, and test name inside the plot
    anova_text = f'ANOVA Test\nF-value: {f_value:.2f}\nP-value: {p_value:.4f}'
    ax.text(
        *text_position, anova_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='bottom', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5)
    )

    # Customize the plot
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Collection Date', fontsize=14)
    ax.set_ylabel('Z-RNA Regions Length', fontsize=14)
    plt.xticks(rotation=90, fontsize=10)

    # Save the figure if specified
    if save_figure:
        plt.savefig(f"{file_name}_boxplot.png", dpi=300)
        plt.savefig(f"{file_name}_boxplot.pdf", dpi=300)

    # Close the figure if specified
    if close_figure:
        plt.close()

    # Show the plot
    plt.show()

def plot_time_length_regression_linages(
    meta_df: pd.DataFrame,
    length_column: str = '',
    category_column: str = 'WHO',
    title: str = '',
    remove_outliers: Tuple[bool, int] = (False, 3),
    figsize: Tuple[int, int] = (10, 5),
    point_size: int = 3,
    save_figure: bool = False,
    normalize: bool = False,
    file_name: str = 'results',
    legend_loc: str = 'best',
    close_figure: bool = False,
    show_legend: bool = True
) -> None:
    """
    Plot a time-length regression of Z-RNA regions' length against the collection date for different lineages.

    Parameters:
    meta_df (pd.DataFrame): The dataframe containing the data to be plotted.
    title (str, optional): The title of the plot. Defaults to 'Time and ZNA length regression'.
    remove_outliers (Tuple[bool, int], optional): A tuple indicating whether to remove outliers
    and the z-score threshold for outlier removal. Defaults to (False, 3).
    figsize (Tuple[int, int], optional): The size of the figure. Defaults to (10, 5).
    point_size (int, optional): The size of the points in the scatter plot. Defaults to 3.
    save_figure (bool, optional): Whether to save the figure as a PNG. Defaults to False.
    normalize (bool, optional): Whether to normalize the data. Defaults to False.
    file_name (str, optional): The name of the file to save the figure as. Defaults to 'results'.
    legend_loc (str, optional): The location of the legend. Defaults to 'best'.
    close_figure (bool, optional): Whether to close the figure after plotting. Defaults to False.

    Returns:
    None
    """

    # Copy the dataframe
    df_copy = meta_df.copy()

    # Normalize the data if specified
    if normalize:
        df_copy['Normalized_Length'] = df_copy['Intervals Total Length'] / df_copy['Sequence Length']
        length_column = 'Normalized_Length'

    # Convert the collection date to datetime and ordinal
    df_copy['Collection_Date'] = pd.to_datetime(df_copy['datetime'], errors='coerce')
    df_copy = df_copy.dropna(subset=['Collection_Date'])
    df_copy['Date_Ordinal'] = df_copy['Collection_Date'].apply(lambda x: x.toordinal())

    # Remove outliers if specified
    if remove_outliers[0]:
        z_scores = df_copy[['Date_Ordinal', 'Intervals Total Length']].apply(
            lambda x: (x - x.mean()) / x.std()
        )
        df_copy = df_copy[
            (np.abs(z_scores['Date_Ordinal']) <= remove_outliers[1]) &
            (np.abs(z_scores['Intervals Total Length']) <= remove_outliers[1])
        ]

    # Create a color map
    unique_values = df_copy[category_column].unique()
    colors = get_high_contrast_colors_gr(len(unique_values))
    color_map = dict(zip(unique_values, colors))

    # Fit a linear regression model
    X = df_copy[['Date_Ordinal']]
    y = df_copy[length_column]
    regression_model = LinearRegression()
    regression_model.fit(X, y)
    y_pred = regression_model.predict(X)

    # Create the figure
    plt.figure(figsize=figsize)

    # Plot the data points for each cluster
    for linage in set(df_copy[category_column]):
        plt.scatter(df_copy[df_copy[category_column] == linage]['Collection_Date'],
                    df_copy[df_copy[category_column] == linage][length_column],
                    color=color_map[linage], label=f"{linage}", s=point_size)

    # Plot the regression line
    plt.plot(df_copy['Collection_Date'], y_pred, color='red', label='Regression Line')

    # Get the slope and intercept of the regression line
    slope = regression_model.coef_[0]
    intercept = regression_model.intercept_

    # Create a list of legend elements
    if show_legend:
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', label=f"{linage}",
                   markerfacecolor=color, markersize=10)
            for linage, color in color_map.items()
        ]
        legend_elements.append(Line2D([0], [0], color='red', lw=2, label='Regression Line'))
        legend_elements.append(Line2D([0], [0], marker='None', color='w',
                                      label=f"Slope = {slope:.2f}\nIntercept = {intercept:.2f}"))
        plt.legend(handles=legend_elements, loc=legend_loc, framealpha=0.5)

    # Customize the plot
    plt.grid(visible=True, which='major', axis='both', linestyle='-')
    plt.title(title)
    plt.xlabel('Collection Date')
    plt.ylabel('Z-RNA Regions Length')

    # Save the figure if specified
    if save_figure:
        plt.savefig(f"{file_name}_heatmap.png")
        plt.savefig(f"{file_name}_heatmap.pdf")

    # Close the figure if specified
    if close_figure:
        plt.close()

    # Show the plot
    plt.show()

# Филогенетический анализ референсных штаммов

In [ ]:
taxa_db.head()

In [ ]:
# Dowload refseq gb files from genbank
%cd /content
!mkdir refseqs

accession_list = taxa_db['Accession']
fetch_gb_files(accession_list, email=Entrez.email, output_dir="/content/refseqs")

In [ ]:
directory = '/content/refseqs'
file_paths = [os.path.join(directory, file)
              for file in os.listdir(directory)
              if os.path.isfile(os.path.join(directory, file))]

protein_names_list = []

for gb_file in file_paths:
  protein_names_list.extend(list_proteins(gb_file))

unique_protein_names = sorted(set(protein_names_list))
print(unique_protein_names)

In [ ]:
# All nucleocapsid protein aliases used in gb annotations
protein_names = ['nucleocapsid',
'nucleocapsid phosphoprotein',
'nucleocapsid protein',
'nucleoprotein',
'N',
'N protein',
'capsid']

In [ ]:
directory = '/content/refseqs'
file_paths = [os.path.join(directory, file)
              for file in os.listdir(directory)
              if os.path.isfile(os.path.join(directory, file))]
seq_names = []
sequences = []
for gb_file in file_paths:
  for pr in protein_names:
    seq = extract_protein_sequence(gb_file, pr)
    if seq:
      seq_names.append(gb_file.split('/')[3][:-3])
      sequences.append(seq)

In [ ]:
# Two species are missing, no protein annotations
missing = [i.split('/')[3][:-3] for i in file_paths if i.split('/')[3][:-3] not in seq_names]
print(missing)

In [ ]:
%cd /content

In [ ]:
# Combine nucleocapsid protein sequences inta a single fasta file
write_to_fasta(seq_names, sequences, 'ref_sequences_nucleocapsid.fas')

In [ ]:
# Multiple sequence alignment
!mafft --auto /content/ref_sequences_nucleocapsid.fas > /content/ref_sequences_nucleocapsid_aln.fas

In [ ]:
# Alignment trimming
!trimal/source/trimal -in /content/ref_sequences_nucleocapsid_aln.fas \
                      -out /content/ref_sequences_nucleocapsid_aln_trimm.fas \
                      -automated1

In [ ]:
# Substitution model estimation and tree building
%cd /content/IQ-TREE/build
!./iqtree -nt AUTO -s /content/ref_sequences_nucleocapsid_aln_trimm.fas -m TEST
%cd /content

# Анализ участков Z-РНК референсных штаммов семейства Coronaviridae

In [ ]:
# Convert and combine all reference sequences' GenBank files into one FASTA file
directory = '/content/refseqs'
file_paths = [os.path.join(directory, file)
              for file in os.listdir(directory)
              if os.path.isfile(os.path.join(directory, file))]
with open('refseqs_all.fas', 'w') as fasta_out:
    for gb_file in file_paths:
        for record in SeqIO.parse(gb_file, "genbank"):
            SeqIO.write(record, fasta_out, "fasta")

In [ ]:
# Predict Z-RNA regions for reference sequences with ZDNABERT
zdnabert_refseq_intervals = process_fasta_with_function('/content/refseqs_all.fas', run_zdnabert, ZDNABERT_PARAMS)

In [ ]:
# Predict Z-RNA regions for reference sequences with ZDNABERT
zhunt_refseq_intervals = process_fasta_with_function('/content/refseqs_all.fas', run_zhunt, ZHUNT_PARAMS)

In [ ]:
# Saving results into a JSON file
with open('refs_zdnabert.json', 'w') as fp:
    json.dump(zdnabert_refseq_intervals, fp, cls=NumpyEncoder)

with open('refs_zhunt.json', 'w') as fp:
    json.dump(zhunt_refseq_intervals, fp, cls=NumpyEncoder)

# Анализ участков Z-РНК для каждого вида вируса из семейства *Coronaviridae*

In [ ]:
# Iterate over unique species and count strains
unique_species = taxa_db["Species"].unique()

data = []
for species in unique_species:
    taxid = get_taxid(species)
    if taxid:
        entry_count = count_genbank_entries(taxid)
        print(f"Found {entry_count} strains for species: {species}.")
    else:
        print(f"No taxid found for species: {species}. Setting virus strains count to 0.")
        entry_count = 0
    data.append((species, entry_count))
    # Respect NCBI's request to not send more than 3 requests per second
    time.sleep(0.35)

# Create a DataFrame with species and their respective virus counts
df = pd.DataFrame(data, columns=["Species", "Virus Count"])
df.to_csv('Spicies_counts.csv')

In [ ]:
# Filter out 'severe acute respiratory syndrome-related coronavirus' from the dataset and fetch others
filtered_taxa_db = taxa_db[taxa_db["Species"] != "Severe acute respiratory syndrome-related coronavirus"][
    "Species"
].unique()

for species in filtered_taxa_db:
    print(f"Processing: {species}")
    taxid = get_taxid(species)
    if taxid:
        print(f"Found taxid: {taxid} for species: {species}")
        fetch_genbank_taxid(taxid, filename=f"/content/species_gb_files/{species.replace(' ', '_')}.gbk", n=1000)
    else:
        print(f"No taxid found for species: {species}")
        time.sleep(0.35)

In [ ]:
# Search for SARS-CoV sequences excluding SARS-CoV-2
query = 'SARS[Organism] NOT SARS-CoV-2[Organism] AND biomol_genomic[prop] AND ("25000"[SLEN] : "35000"[SLEN]'
handle = Entrez.esearch(db="nucleotide", term=query, retmax=10000)  # Set retmax to the desired number of results
record = Entrez.read(handle)
id_list = record["IdList"]

# Fetch the sequences
sequences = []
for seq_id in tqdm(id_list):
    fetch_handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
    sequence = SeqIO.read(fetch_handle, "genbank")
    sequences.append(sequence)
    fetch_handle.close()

# Save sequences to a file
SeqIO.write(sequences,
            "/content/species_gb_files/Severe_acute_respiratory_syndrome_related_coronavirus.gbk",
            "genbank")

In [ ]:
# Search for Human coronavirus OC43
query = 'Human coronavirus OC43[Organism] AND biomol_genomic[prop] AND ("25000"[SLEN] : "35000"[SLEN]'
handle = Entrez.esearch(db="nucleotide", term=query, retmax=10000)  # Set retmax to the desired number of results
record = Entrez.read(handle)
id_list = record["IdList"]

# Fetch the sequences
sequences = []
for seq_id in tqdm(id_list):
    fetch_handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
    sequence = SeqIO.read(fetch_handle, "genbank")
    sequences.append(sequence)
    fetch_handle.close()

# Save sequences to a file
SeqIO.write(sequences,
            "/content/species_gb_files/Human_coronavirus_OC43.gbk",
            "genbank")

In [ ]:
!zip -r /content/species_gb_files.zip /content/species_gb_files
files.download("/content/species_gb_files.zip")

In [ ]:
!mkdir /content/species_fasta_files

In [ ]:
# Convert and combine GenBank files into FASTA files
file_paths = list_files('/content/species_gb_files/')

for gb_file in file_paths:
    virus_name = gb_file.split('/')[3][:-4]
    with open(f'/content/species_fasta_files/{virus_name}.fas', 'w') as fasta_out:
        for record in SeqIO.parse(gb_file, "genbank"):
            SeqIO.write(record, fasta_out, "fasta")

In [ ]:
!mkdir '/content/Coronaviridae strains predictions json'

In [ ]:
# Analyze Z-RNA regions and save results
file_paths = list_files('/content/species_fasta_files/')

for file in file_paths:
    virus_name = file.split('/')[3][:-4]
    zdnabert_refseq_intervals = process_fasta_with_function(file, run_zdnabert, ZDNABERT_PARAMS)
    with open(f"/content/species_json_dnabert/zdnabert_{virus_name}.json", 'w') as results_file:
        json.dump(zdnabert_refseq_intervals, results_file, cls=NumpyEncoder)

    zhunt_refseq_intervals = process_fasta_with_function(file, run_zhunt, ZHUNT_PARAMS)
    with open(f"/content/species_json_zhunt/zhunt_{virus_name}.json", 'w') as results_file:
        json.dump(zdnabert_refseq_intervals, results_file, cls=NumpyEncoder)

In [ ]:
# Download json files
!zip -r /content/species_json_dnaber.zip /content/species_json_dnabert
files.download("/content/species_json_dnaber.zip")

!zip -r /content/species_json_zhunt.zip /content/species_json_zhunt
files.download("/content/species_json_zhunt.zip")

# Визуализация филогенетического дерева референсных штаммов
### с указанием суммарной длины участков Z-РНК для ветвей

In [ ]:
tree_newick_string = open('/content/ref_sequences_nucleocapsid_aln_trimm.fas.treefile', 'r').read().strip()

In [ ]:
length_of_intervals_zdnabert = total_interval_length(zdnabert_refseq_intervals)
length_of_intervals_zhunt = total_interval_length(zhunt_refseq_intervals)

In [ ]:
plot_tree_with_annotations(tree_newick_string, length_of_intervals_zdnabert, taxa_db, color_threshold=0);

In [ ]:
plot_tree_with_annotations(tree_newick_string, length_of_intervals_zhunt, taxa_db, color_threshold=0)

# Подготовка метаданных для последовательностей и участков Z-РНК

## Данные по кластеризации участков Z-РНК

In [ ]:
!mkdir /content/clustered_data_frames_dnabert/
!mkdir /content/clustered_data_frames_zhunt/
!mkdir /content/row_linkage_matrices_dnabert/
!mkdir /content/row_linkage_matrices_zhunt/

In [ ]:
file_paths_zdnabert = sorted(list_files('/content/Coronaviridae strains predictions json/species_json_dnabert'))
file_paths_zhunt = sorted(list_files('/content/Coronaviridae strains predictions json/species_json_zhunt'))

for i, j in list(zip(file_paths_zdnabert, file_paths_zhunt)):
    virus_name = j.split('_', 3)[3][:-5]

    dnabert_intervals = read_json_file(i)
    zhunt_intervals = read_json_file(j)

    dnabert_cls_df, dnabert_rowl = create_clustered_dataframe(dnabert_intervals)
    zhunt_cls_df, zhunt_rowl = create_clustered_dataframe(zhunt_intervals)

    dnabert_cls_df.to_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus_name}.csv')
    zhunt_cls_df.to_csv(f'/content/clustered_data_frames_zhunt/zhunt_cls_df_{virus_name}.csv')

    np.save(f"/content/row_linkage_matrices_dnabert/dnabert_rowl_{virus_name}", dnabert_rowl)
    np.save(f"/content/row_linkage_matrices_zhunt/zhunt_rowl_{virus_name}", zhunt_rowl)

In [ ]:
dnabert_intervals = read_json_file('/content/species_json_dnabert/zdnabert_Human_coronavirus_OC43.json')
zhunt_intervals = read_json_file('/content/species_json_zhunt/zhunt_Human_coronavirus_OC43.json')

dnabert_cls_df, dnabert_rowl = create_clustered_dataframe(dnabert_intervals)
zhunt_cls_df, zhunt_rowl = create_clustered_dataframe(zhunt_intervals)

dnabert_cls_df.to_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_Human_coronavirus_OC43.csv')
zhunt_cls_df.to_csv(f'/content/clustered_data_frames_zhunt/zhunt_cls_df_Human_coronavirus_OC43.csv')

np.save(f"/content/row_linkage_matrices_dnabert/dnabert_rowl_Human_coronavirus_OC43", dnabert_rowl)
np.save(f"/content/row_linkage_matrices_zhunt/zhunt_rowl_Human_coronavirus_OC43", zhunt_rowl)

In [ ]:
# Download csv files
!zip -r /content/clustered_data_frames_dnabert.zip /content/clustered_data_frames_dnabert
files.download("/content/clustered_data_frames_dnabert.zip")
!zip -r /content/clustered_data_frames_zhunt.zip /content/clustered_data_frames_zhunt
files.download("/content/clustered_data_frames_zhunt.zip")

!zip -r /content/row_linkage_matrices_dnabert.zip /content/row_linkage_matrices_dnabert
files.download("/content/row_linkage_matrices_dnabert.zip")
!zip -r /content/row_linkage_matrices_zhunt.zip /content/row_linkage_matrices_zhunt
files.download("/content/row_linkage_matrices_zhunt.zip")

In [ ]:
viruses = [
    'Severe_acute_respiratory_syndrome_related_coronavirus',
    'Middle_East_respiratory_syndrome-related_coronavirus',
    'Human_coronavirus_229E',
    'Human_coronavirus_HKU1',
    'Human_coronavirus_NL63',
    'Human_coronavirus_OC43',
    'Betacoronavirus_1',
    'Bat_coronavirus_HKU10',
    'Avian_coronavirus',
    'Porcine_epidemic_diarrhea_virus',
    'Alphacoronavirus_1',
    'Coronavirus_HKU15',
    'Murine_coronavirus',
    'Rousettus_bat_coronavirus_HKU9'
]

In [ ]:
# Iterate over all viruses and calculate color threshold for each virus
color_thresholds_dnabert = {}

for virus_name in viruses:
    file_path_rowl = f'/content/row_linkage_matrices_dnabert/dnabert_rowl_{virus_name}.npy'
    row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
    color_threshold = optimal_dendrogram_threshold_and_clusters(row_linkage_matrix, 5)
    color_thresholds_dnabert[virus_name] = color_threshold
save_json(color_thresholds_dnabert, 'color_thresholds_dnabert.json')

In [ ]:
# Iterate over all viruses and calculate color threshold for each virus
color_thresholds_zhunt = {}

for virus_name in viruses:
    file_path_rowl = f'/content/row_linkage_matrices_zhunt/zhunt_rowl_{virus_name}.npy'
    row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
    color_threshold = optimal_dendrogram_threshold_and_clusters(row_linkage_matrix, 4)
    color_thresholds_zhunt[virus_name] = color_threshold
save_json(color_thresholds_zhunt, 'color_thresholds_zhunt.json')

## Данные по характеристике участков Z-РНК и геномных последовательностей

In [ ]:
# # Create a dataframe with data from genbank files
# files = list_files('/content/species_gb_files')
# meta_data_all_species = extract_genbank_data(files)

In [ ]:
meta_data_all_species = pd.read_csv('meta_data_all_species.csv', index_col=0)
meta_data_all_species = meta_data_all_species.drop('Clusters', axis=1)

In [ ]:
sars2_meta = pd.read_csv('sars2_meta_10k.csv', index_col=0)
sars2_meta['virus_species_from_file'] = 'Severe_acute_respiratory_syndrome_related_coronavirus_2'

In [ ]:
# Add Z-RNA intervals for all species information to the metadata dataframe

# Combine DNABERT prediction files into one dictionary
combined_dict_dnabert = {}
for file_path in list_files('/content/Coronaviridae strains predictions json/species_json_dnabert'):
    with open(file_path, 'r') as f:
        combined_dict_dnabert.update(json.load(f))

# Update the metadata dataframe with DNABERT information
meta_data_all_species['ZDNABERT intervals total length'] = meta_data_all_species['accession_with_version'].apply(
    lambda x: sum_intervals(combined_dict_dnabert.get(x))
)
meta_data_all_species['ZDNABERT intervals number'] = meta_data_all_species['accession_with_version'].apply(
    lambda x: len(combined_dict_dnabert.get(x))
)

# Combine ZHUNT prediction files into one dictionary
combined_dict_zhunt = {}
for file_path in list_files('/content/Coronaviridae strains predictions json/species_json_zhunt'):
    with open(file_path, 'r') as f:
        combined_dict_zhunt.update(json.load(f))

# Update the metadata dataframe with ZHUNT information
meta_data_all_species['ZHUNT intervals total length'] = meta_data_all_species['accession_with_version'].apply(
    lambda x: sum_intervals(combined_dict_zhunt.get(x, None))
)
meta_data_all_species['ZHUNT intervals number'] = meta_data_all_species['accession_with_version'].apply(
    lambda x: len(combined_dict_zhunt.get(x, None))
)

In [ ]:
# Add SARS-CoV-2 Z-RNA intervals for all species information to the metadata dataframe

# Combine DNABERT prediction files into one dictionary
combined_sars2_dnabert = {}
for file_path in list_files('/content/SARS-CoV-2-parts/DNABERT'):
    with open(file_path, 'r') as f:
        combined_sars2_dnabert.update(json.load(f))

# Update the metadata dataframe with DNABERT information
sars2_meta['ZDNABERT intervals total length'] = sars2_meta['accession_with_version'].apply(
    lambda x: sum_intervals(combined_sars2_dnabert.get(x))
)
sars2_meta['ZDNABERT intervals number'] = sars2_meta['accession_with_version'].apply(
    lambda x: len(combined_sars2_dnabert.get(x))
)

# Combine ZHUNT prediction files into one dictionary
combined_sars2_zhunt = {}
for file_path in list_files('/content/SARS-CoV-2-parts/ZHUNT'):
    with open(file_path, 'r') as f:
        combined_sars2_zhunt.update(json.load(f))

# Update the metadata dataframe with ZHUNT information
sars2_meta['ZHUNT intervals total length'] = sars2_meta['accession_with_version'].apply(
    lambda x: sum_intervals(combined_sars2_zhunt.get(x))
)
sars2_meta['ZHUNT intervals number'] = sars2_meta['accession_with_version'].apply(
    lambda x: len(combined_sars2_zhunt.get(x))
)

In [ ]:
joint_meta_df = pd.concat([meta_data_all_species, sars2_meta], ignore_index=True)

In [ ]:
joint_meta_df['ZDNABERT Mean interval length'] = (
    joint_meta_df['ZDNABERT intervals total length'] /
    joint_meta_df['ZDNABERT intervals number']
)

joint_meta_df['ZHUNT Mean interval length'] = (
    joint_meta_df['ZHUNT intervals total length'] /
    joint_meta_df['ZHUNT intervals number']
)

In [ ]:
# Update the metadata dataframe with Host information
joint_meta_df['Host'] = joint_meta_df['virus_species_from_file'].apply(
    lambda x: PREVALENT_HOSTS.get(x, None))

In [ ]:
# Manually fix some NA values
joint_meta_df.loc[joint_meta_df['virus_species_from_file'] == 'Eidolon_bat_coronavirus_C704', 'genus'] = 'Betacoronavirus'
joint_meta_df.loc[joint_meta_df['virus_species_from_file'] == 'Myodes_coronavirus_2JL14', 'genus'] = 'Betacoronavirus'

In [ ]:
# Add 'Virus name' column, just for aestetics
joint_meta_df['Virus name'] = joint_meta_df['virus_species_from_file'].apply(lambda x: x.replace('_', ' '))

In [ ]:
joint_meta_df.to_csv('joint_meta_df.csv')

In [ ]:
joint_meta_df = pd.read_csv('joint_meta_df.csv')

In [ ]:
joint_meta_df = remove_outliers(joint_meta_df, ['ZDNABERT intervals total length'], multiplier = 15)
#joint_meta_df = remove_outliers(joint_meta_df, ['GC content'], multiplier = 10)

# Визуализация распределения характеристик геномной последовательности в зависимости от вида вируса (с указанием основного хозяина вируса)

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'Sequence length',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'GC content',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'GC skew',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

# Визуализация распределения характеристик участков Z-РНК в зависимости от вида вируса (с указанием основного хозяина вируса)

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'ZDNABERT intervals total length',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'ZHUNT intervals total length',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'ZDNABERT intervals number',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'ZHUNT intervals number',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'ZDNABERT Mean interval length',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

In [ ]:
draw_boxplot_species(
        data_subset = joint_meta_df,
        category_label = 'Virus name',
        value_label = 'ZHUNT Mean interval length',
        figure_size = (30, 10),
        text_position = (0.006, 0.46),
        legend_location = 'upper left'
        )

# Визуализация взаимного расположения участков Z-РНК предсказанных ZDNABERT и ZHUNT


In [ ]:
genomic_regions_sars = extract_regions_from_genbank_id('NC_004718.3')

In [ ]:
genomic_regions_sars = [
    ("5'UTR", 1, 264), ('nsp1', 265, 804), ('nsp2', 805, 2718),
    ('nsp3', 2719, 8484), ('nsp4', 8485, 9984), ('nsp5', 9985, 10902),
    ('nsp6', 10903, 11772), ('nsp7', 11773, 12021), ('nsp8', 12022, 12615),
    ('nsp9', 12616, 12954), ('nsp10', 12955, 13371), ('nsp11', 13372, 13394),
    ('nsp12', 13372, 16166), ('nsp13', 16167, 17969), ('nsp14', 17970, 19550),
    ('nsp15', 19551, 20588), ('nsp16', 20589, 21482), ('S', 21492, 25259),
    ('ORF3a', 25268, 26092), ('ORF3b', 25689, 26153), ('E', 26117, 26347),
    ('M', 26398, 27063), ('ORF6', 27074, 27265), ('ORF7a', 27273, 27641),
    ('ORF7b', 27638, 27772), ('ORF8a', 27779, 27898), ('ORF8b', 27864, 28118),
    ('N', 28120, 29388), ('ORF9b', 28130, 28426), ('ORF9a', 28583, 28795),
    ("3'UTR", 28796, 29751)
]

In [ ]:
# SARS-CoV
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Severe_acute_respiratory_syndrome_related_coronavirus'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions("SARS", sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_sars, colors)

In [ ]:
genomic_regions_MERS = extract_regions_from_genbank_id('NC_019843.3')

In [ ]:
genomic_regions_MERS = [
    ("5'UTR", 1, 278), ('nsp1', 279, 857), ('nsp2', 858, 2837),
    ('nsp3', 2838, 8498), ('nsp4', 8499, 10019), ('nsp5', 10020, 10937),
    ('nsp6', 10938, 11813), ('nsp7', 11814, 12062), ('nsp8', 12063, 12659),
    ('nsp9', 12660, 12989), ('nsp10', 12990, 13409), ('nsp11', 13410, 16207),
    ('nsp12', 16208, 18001), ('nsp13', 18002, 19573), ('nsp14', 19574, 20602),
    ('nsp15', 20603, 21511), ('S', 21456, 25517), ('NS3', 25532, 25843),
    ('NS4A', 25852, 26181), ('NS4B', 26093, 26833), ('NS5', 26840, 27514),
    ('E', 27590, 27838), ('M', 27853, 28512), ('N', 28566, 29807),
    ('ORF8b', 28762, 29100), ("3'UTR", 29101, 30119)
]

In [ ]:
# MERS
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Middle_East_respiratory_syndrome-related_coronavirus'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions("MERS", sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_MERS, colors)

In [ ]:
genomic_regions_229E = extract_regions_from_genbank_id('NC_002645.1')

In [ ]:
genomic_regions_229E = [
    ("5'UTR", 1, 292), ('leader protein p9', 293, 625), ('p87', 626, 2983),
    ('nsp1', 2984, 7744), ('p44', 7745, 9187), ('nsp2', 9188, 10093),
    ('nsp3', 10094, 10930), ('nsp4', 10931, 11179), ('nsp5', 11180, 11764),
    ('nsp6', 11765, 12091), ('nsp7', 12092, 12496), ('nsp9', 12497, 15276),
    ('nsp10', 15277, 17067), ('nsp11', 17068, 18621), ('nsp12', 18622, 19665),
    ('nsp13', 19666, 20565), ('S', 20570, 24091), ('4a', 24091, 24492),
    ('4bn', 24482, 24748), ('E', 24750, 24983), ('M', 24995, 25672),
    ('N', 25686, 26855), ("3'UTR", 26856, 27317)
]

In [ ]:
# 229E
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Human_coronavirus_229E'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions("Human coronavirus 229E", sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_229E, colors)

In [ ]:
genomic_regions_Human_HKU1 = extract_regions_from_genbank_id('NC_006577.2')

In [ ]:
genomic_regions_Human_HKU1 = [
    ("5'UTR", 1, 205), ('Leader protein', 206, 871), ('nsp2', 872, 2632),
    ('nsp3', 2633, 8719), ('nsp4', 8720, 10207), ('nsp5', 10208, 11116),
    ('nsp6', 11117, 11977), ('nsp7', 11978, 12253), ('nsp8', 12254, 12835),
    ('nsp9', 12836, 13165), ('nsp10', 13166, 13576), ('nsp12', 13577, 16359),
    ('nsp11', 13577, 13618), ('nsp13', 16360, 18168), ('nsp14', 18169, 19731),
    ('nsp15', 19732, 20853), ('nsp16', 20854, 21750), ('HE', 21773, 22933),
    ('S', 22942, 27012), ('NS', 27051, 27380), ('E', 27373, 27621),
    ('M', 27633, 28304), ('N', 28320, 29645), ('N2', 28342, 28959),
    ("3'UTR", 28960, 29926)
]

In [ ]:
# Human_HKU1
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Human_coronavirus_HKU1'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Human_HKU1, colors)

In [ ]:
genomic_regions_Human_NL63 = extract_regions_from_genbank_id('NC_005831.2')

In [ ]:
genomic_regions_Human_NL63 = [
    ("5'UTR", 1, 286), ("ORF1ab", 287, 20475), ('S', 20472, 24542),
    ('p3', 24542, 25219), ('E', 25200, 25433), ('M', 25442, 26122),
    ('N', 26133, 27266), ("3'UTR", 27267, 27553)
]

In [ ]:
# Human_NL63
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Human_coronavirus_NL63'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Human_NL63, colors)

In [ ]:
genomic_regions_Human_OC43 = extract_regions_from_genbank_id('NC_006213.1')

In [ ]:
genomic_regions_Human_OC43 = [
    ("5'UTR", 1, 209), ('nsp1', 210, 947), ('nsp2', 948, 2762),
    ('nsp3', 2763, 8459), ('nsp4', 8460, 9947), ('nsp5', 9948, 10856),
    ('nsp6', 10857, 11717), ('nsp7', 11718, 11984), ('nsp8', 11985, 12575),
    ('nsp9', 12576, 12905), ('nsp10', 12906, 13316), ('nsp11', 13317, 13358),
    ('nsp12', 13317, 16099), ('nsp13', 16100, 17908), ('nsp14', 17909, 19471),
    ('nsp15', 19472, 20596), ('nsp16', 20597, 21493), ('NS2', 21506, 22342),
    ('HE', 22354, 23628), ('S', 23643, 27704), ('ORF5', 27792, 28121),
    ('E', 28108, 28362), ('M', 28377, 29069), ('N', 29079, 30425),
    ('I', 29140, 29322), ("3'UTR", 30424, 30741)
]

In [ ]:
# Human coronavirus OC43
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Human_coronavirus_OC43'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv('/content/clustered_data_frames_dnabert/dnabert_cls_df_Human_coronavirus_OC43.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Human_OC43, colors)

In [ ]:
# Same as for Human coronavirus OC43
genomic_regions_Betacoronavirus1 = extract_regions_from_genbank_id('NC_006213.1')

In [ ]:
genomic_regions_Betacoronavirus1 = [
    ("5'UTR", 1, 209), ('nsp1', 210, 947), ('nsp2', 948, 2762),
    ('nsp3', 2763, 8459), ('nsp4', 8460, 9947), ('nsp5', 9948, 10856),
    ('nsp6', 10857, 11717), ('nsp7', 11718, 11984), ('nsp8', 11985, 12575),
    ('nsp9', 12576, 12905), ('nsp10', 12906, 13316), ('nsp11', 13317, 13358),
    ('nsp12', 13317, 16099), ('nsp13', 16100, 17908), ('nsp14', 17909, 19471),
    ('nsp15', 19472, 20596), ('nsp16', 20597, 21493), ('NS2', 21506, 22342),
    ('HE', 22354, 23628), ('S', 23643, 27704), ('ORF5', 27792, 28121),
    ('E', 28108, 28362), ('M', 28377, 29069), ('N', 29079, 30425),
    ('I', 29140, 29322), ("3'UTR", 30424, 30741)
]

In [ ]:
# Betacoronavirus 1
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Betacoronavirus_1'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Betacoronavirus1, colors)

In [ ]:
genomic_regions_Bat_HKU10 = extract_regions_from_genbank_id('NC_018871.1')

In [ ]:
genomic_regions_Bat_HKU10 = [
    ("5'UTR", 1, 302), ("ORF1ab", 303, 20644), ('S', 20641, 24690),
    ('NS3', 24690, 25346), ('E', 25375, 25602), ('M', 25608, 26297),
    ('N', 26308, 27516), ('NS7a', 27532, 27777), ('NS7b', 27787, 28248),
    ('NS7c', 27986, 28216), ("3'UTR", 28249, 28494)
]

In [ ]:
# Bat coronavirus HKU10
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Bat_coronavirus_HKU10'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Bat_HKU10, colors)

In [ ]:
genomic_regions_Avian = extract_regions_from_genbank_id('NC_048214')

In [ ]:
genomic_regions_Avian = [
    ("5'UTR", 1, 346), ('nsp1/2', 347, 2356), ('nsp3', 2357, 7225),
    ('nsp4', 7226, 8770), ('nsp5', 8771, 9691), ('nsp6', 9692, 10573),
    ('nsp7', 10574, 10822), ('nsp8', 10823, 11452), ('nsp9', 11453, 11785),
    ('nsp10', 11786, 12220), ('nsp12', 12221, 15081), ('nsp11', 12221, 12286),
    ('nsp13', 15082, 16881), ('nsp14', 16882, 18447), ('nsp15', 18448, 19461),
    ('nsp16', 19462, 20292), ('S', 20318, 23893), ('3a', 23893, 24066),
    ('3b', 24066, 24257), ('3c', 24238, 24519), ('M', 24516, 25223),
    ('ORFX', 25224, 25508), ('5a', 25587, 25784), ('5b', 25781, 26029),
    ('N', 25972, 27216), ('ORFY', 27246, 27458), ("3'UTR", 27459, 27754)
]

In [ ]:
# Avian coronavirus
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Avian_coronavirus'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Avian, colors)

In [ ]:
genomic_regions_Porcine_ep = extract_regions_from_genbank_id('KJ184549.1')

In [ ]:
genomic_regions_Porcine_ep = [
    ("5'UTR", 1, 292), ('nsp1', 293, 622), ('nsp2', 623, 2872),
    ('nsp3', 2873, 7840), ('nsp4', 7841, 9265), ('nsp4', 7841, 9283),
    ('nsp5', 9284, 10189), ('nsp6', 10190, 11029), ('nsp7', 11030, 11281),
    ('nsp8', 11282, 11863), ('nsp9', 11864, 12187), ('nsp10', 12188, 12592),
    ('nsp11', 12593, 12643), ('nsp12', 12593, 15372), ('nsp13', 15373, 17163),
    ('nsp14', 17164, 18714), ('nsp15', 18715, 19731), ('nsp16', 19732, 20634),
    ('S', 20634, 24794), ('ORF3', 24794, 25468), ('E', 25449, 25679),
    ('M', 25687, 26367), ('N', 26379, 27704), ("3'UTR", 27705, 28038)
]

In [ ]:
# Porcine epidemic diarrhea virus
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Porcine_epidemic_diarrhea_virus'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Porcine_ep, colors)

In [ ]:
genomic_regions_Alphacoronavirus_1 = extract_regions_from_genbank_id('MT239439.1')

In [ ]:
genomic_regions_Alphacoronavirus_1 = [
    ("5'UTR", 1, 309), ('nsp1', 310, 639), ('nsp2', 640, 2946),
    ('nsp3', 2947, 7515), ('nsp4', 7516, 8985), ('nsp5', 8986, 9891),
    ('nsp6', 9892, 10773), ('nsp7', 10774, 11022), ('nsp8', 11023, 11607),
    ('nsp9', 11608, 11940), ('nsp10', 11941, 12345), ('nsp11', 12346, 12402),
    ('nsp12', 12403, 15131), ('nsp13', 15132, 16928), ('nsp14', 16929, 18485),
    ('nsp15', 18486, 19502), ('nsp16', 19503, 20402), ('S', 20402, 24766),
    ('3a', 24851, 25066), ('3b', 25011, 25226), ('E', 25935, 26183),
    ('M', 26194, 26982), ('N', 26995, 28128), ('7a', 28133, 28438),
    ('7b', 28443, 29063), ("3'UTR", 29063, 29347)
]

In [ ]:
# Alphacoronavirus 1
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Alphacoronavirus_1'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Alphacoronavirus_1, colors)

In [ ]:
genomic_regions_Coronavirus_HKU15 = extract_regions_from_genbank_id('NC_039208.1')

In [ ]:
genomic_regions_Coronavirus_HKU15 = [
    ("5'UTR", 1, 532), ("ORF1ab", 533, 19316), ('S', 19317, 22796),
    ('E', 22797, 23041), ('M', 23042, 23687), ('NS6 protein', 23688, 23971),
    ('N', 23987, 24085), ('NS7 protein', 24086, 25020), ("3'UTR", 25021, 25425)
]

In [ ]:
# Coronavirus HKU15
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Coronavirus_HKU15'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Coronavirus_HKU15, colors)

In [ ]:
genomic_regions_Murine_coronavirus = extract_regions_from_genbank_id('NC_001846.1')

In [ ]:
genomic_regions_Murine_coronavirus = [
    ("5'UTR", 1, 209), ('nsp1', 210, 950), ('nsp2', 951, 2705),
    ('nsp3', 2706, 9632), ('nsp4', 9633, 10208), ('nsp5', 10209, 11117),
    ('nsp6', 11118, 11978), ('nsp7', 11979, 12245), ('nsp8', 12246, 12836),
    ('nsp9', 12837, 13166), ('nsp10', 13167, 13577), ('nsp11', 13578, 13619),
    ('nsp12', 13578, 16360), ('nsp13', 16361, 18160), ("nsp14", 18161, 19723),
    ('nsp15', 19724, 20845), ("nsp16", 20846, 21742), ('NS2a', 21771, 22556),
    ('E2', 23929, 27903), ('E', 28706, 28957), ('E1', 28968, 29654),
    ('N', 29669, 31033), ('I protein', 29733, 30356), ("3'UTR", 31032, 31357)
]

In [ ]:
# Murine coronavirus
pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Murine_coronavirus'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Murine_coronavirus, colors)

In [ ]:
genomic_regions_Rousettus_HKU9 = extract_regions_from_genbank_id('NC_009021.1')

In [ ]:
genomic_regions_Rousettus_HKU9 = [
    ("5'UTR", 1, 228), ('nsp1', 229, 753), ('nsp2', 754, 2544),
    ('nsp3', 2545, 8055), ('nsp4', 8056, 9537), ('nsp5', 9538, 10455),
    ('nsp6', 10456, 11325), ('nsp7', 11326, 11574), ('nsp8', 11575, 12174),
    ('nsp9', 12175, 12510), ('nsp10', 12511, 12927), ('nsp11', 12928, 15722),
    ('nsp12', 15723, 17525), ('nsp13', 17526, 19115), ('nsp14', 19116, 20126),
    ("nsp15", 20127, 21017), ('S', 20974, 24798), ('NS3', 24795, 25457),
    ('E', 25457, 25696), ('M', 25689, 26357), ('N', 26419, 27825),
    ('NS7a', 27869, 28426), ('NS7b', 28433, 28882), ("3'UTR", 28883, 29114)
]

In [ ]:
# Rousettus bat coronavirus HKU9

pref = '/content/Coronaviridae strains predictions json/species_json_'

virus = 'Rousettus_bat_coronavirus_HKU9'

dnabert_intervals = read_json_file(f"{pref}dnabert/zdnabert_{virus}.json")
zhunt_intervals = read_json_file(f"{pref}zhunt/zhunt_{virus}.json")

dnabert_cls_df = pd.read_csv(f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus}.csv')

sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions(virus.replace('_', ' '), sorted_keys,
                 dnabert_intervals,
                 zhunt_intervals,
                 genomic_regions_Rousettus_HKU9, colors)

# Визуализация кластеров участков Z-РНК предсказанных ZDNABERT и ZHUNT

In [ ]:
viruses = [
    'Severe_acute_respiratory_syndrome_related_coronavirus',
    'Middle_East_respiratory_syndrome-related_coronavirus',
    'Human_coronavirus_229E',
    'Human_coronavirus_HKU1',
    'Human_coronavirus_NL63',
    'Human_coronavirus_OC43',
    'Betacoronavirus_1',
    'Bat_coronavirus_HKU10',
    'Avian_coronavirus',
    'Porcine_epidemic_diarrhea_virus',
    'Alphacoronavirus_1',
    'Coronavirus_HKU15',
    'Murine_coronavirus',
    'Rousettus_bat_coronavirus_HKU9'
]

In [ ]:
# Suppress the specific warning from scipy
warnings.filterwarnings("ignore", category=UserWarning, module="scipy.cluster.hierarchy")

# Iterate over all viruses and plot clusters dendrogram and heatmap for each virus

for virus_name in viruses:
    try:
        file_path_df = f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus_name}.csv'
        file_path_rowl = f'/content/row_linkage_matrices_dnabert/dnabert_rowl_{virus_name}.npy'

        clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
        row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)

        title = f"{virus_name.replace('_', ' ')} (ZDNABERT)"

        color_threshold = color_thresholds_dnabert[virus_name]

        plot_heatmap_with_dendrogram(clustered_dataframe, row_linkage_matrix, color_threshold,
                                     title=title, figsize=(10, 10), save_figure=True, file_name=virus_name)
    except Exception as e:
        print(f"Error processing {virus_name}: {e}")
        continue

In [ ]:
# Iterate over all viruses and plot clusters dendrogram and heatmap for each virus

for virus_name in viruses:
    try:
        file_path_df = f'/content/clustered_data_frames_zhunt/zhunt_cls_df_{virus_name}.csv'
        file_path_rowl = f'/content/row_linkage_matrices_zhunt/zhunt_rowl_{virus_name}.npy'

        clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
        row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)

        title = f"{virus_name.replace('_', ' ')} (zhunt)"

        color_threshold = color_thresholds_zhunt[virus_name]

        plot_heatmap_with_dendrogram(clustered_dataframe, row_linkage_matrix, color_threshold,
                                     title=title, figsize=(10, 10), save_figure=True, file_name=virus_name)
    except Exception as e:
        print(f"Error processing {virus_name}: {e}")
        continue

# Анализ корреляций

In [ ]:
meta_data_all_species = pd.read_csv('meta_data_all_species.csv', index_col=0)
meta_data_all_species = meta_data_all_species.drop(['Melting temperature', 'Clusters'], axis=1)

In [ ]:
meta_data_all_species.shape

In [ ]:
column_names = [
    'GC content', 'GC skew', 'ZDNABERT intervals total length',
    'ZHUNT intervals total length', 'Collection date'
]

columns_with_outliers = [
    'ZDNABERT intervals total length', 'GC content', 'GC skew', 'genus'
]

meta_data_all_species = remove_outliers(
    meta_data_all_species,
    column_names=['ZDNABERT intervals total length'],
    multiplier=3
)

# Convert 'datetime' column to datetime objects
meta_data_all_species['datetime'] = pd.to_datetime(meta_data_all_species['datetime'])

# Only convert non-NaT values to ordinal, leave NaT values as NaN
meta_data_all_species['Collection date'] = meta_data_all_species['datetime'].apply(
    lambda x: x.toordinal() if pd.notna(x) else np.nan
)

meta_data_all_species.shape

In [ ]:
# Compute the correlation matrix
corr = meta_data_all_species.corr(numeric_only=True)

# Generate a mask for the upper triangle (optional)
mask = None
upper_triangle = True
if upper_triangle:
    mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Generate the heatmap
sns.heatmap(corr, mask=mask, cmap="coolwarm", vmax=1, vmin=-1, center=0, annot=True);

In [ ]:
# Set the font scale and markers
sns.set_context("notebook", font_scale=0.6)
markers = ["x", "+", "1", "2", "3"]

In [ ]:
selected_columns_seq_prop = ['Sequence length', 'GC content', 'GC skew', 'genus']
sns.pairplot(meta_data_all_species[selected_columns_seq_prop], diag_kind="kde", hue="genus", markers=markers);

In [ ]:
x_vars = ['Sequence length', 'GC content', 'GC skew']
y_vars = ['ZDNABERT intervals total length']
sns.pairplot(meta_data_all_species, x_vars=x_vars, y_vars=y_vars, diag_kind="kde", hue="genus", markers=markers);

In [ ]:
x_vars = ['Sequence length', 'GC content', 'GC skew']
y_vars = ['ZDNABERT intervals number']
sns.pairplot(meta_data_all_species, x_vars=x_vars, y_vars=y_vars, diag_kind="kde", hue="genus", markers=markers);

In [ ]:
x_vars = ['Sequence length', 'GC content', 'GC skew']
y_vars = ['ZHUNT intervals total length']
sns.pairplot(meta_data_all_species, x_vars=x_vars, y_vars=y_vars, diag_kind="kde", hue="genus", markers=markers);

In [ ]:
x_vars = ['Sequence length', 'GC content', 'GC skew']
y_vars = ['ZHUNT intervals number']
sns.pairplot(meta_data_all_species, x_vars=x_vars, y_vars=y_vars, diag_kind="kde", hue="genus", markers=markers);

In [ ]:
z = [
 'ZDNABERT intervals total length',
 'ZDNABERT intervals number',
 'ZHUNT intervals total length',
 'ZHUNT intervals number', 'genus'
 ]

In [ ]:
sns.pairplot(meta_data_all_species[z], diag_kind="kde", hue="genus", markers=markers);

# Регрессионный анализ зависимости суммарной длины участков Z-РНК от времени сбора образца вируса

In [ ]:
meta_data_all_species = pd.read_csv('meta_data_all_species.csv', index_col=0)

In [ ]:
viruses = [
    'Severe_acute_respiratory_syndrome_related_coronavirus',
    'Middle_East_respiratory_syndrome-related_coronavirus',
    'Human_coronavirus_229E',
    'Human_coronavirus_HKU1',
    'Human_coronavirus_NL63',
    'Human_coronavirus_OC43',
    'Betacoronavirus_1',
    'Bat_coronavirus_HKU10',
    'Avian_coronavirus',
    'Porcine_epidemic_diarrhea_virus',
    'Alphacoronavirus_1',
    'Coronavirus_HKU15',
    'Murine_coronavirus',
    'Rousettus_bat_coronavirus_HKU9'
]

In [ ]:
# Iterate over all viruses and plot intervals total length vs date for each virus ZDNABERT predictions

for virus_name in viruses:
    virus = virus_name.replace('_', ' ')
    file_path_df = f'/content/clustered_data_frames_dnabert/dnabert_cls_df_{virus_name}.csv'
    file_path_rowl = f'/content/row_linkage_matrices_dnabert/dnabert_rowl_{virus_name}.npy'

    clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
    row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
    color_threshold = color_thresholds_dnabert[virus_name]

    dendro = plot_time_length_regression(meta_df=meta_data_all_species,
                            virus_name=virus_name,
                            clustered_dataframe=clustered_dataframe,
                            row_linkage_matrix=row_linkage_matrix,
                            color_threshold = color_threshold,
                            point_size = 20,
                            intervals_column='ZDNABERT intervals total length',
                            title=f'{virus}\nZDNABERT predictions',
                            figsize=(15, 10), remove_outliers=(True, 3))

In [ ]:
# Iterate over all viruses and plot intervals total length vs date for each virus ZHUNT predictions

for virus_name in viruses:
    virus = virus_name.replace('_', ' ')
    file_path_df = f'/content/clustered_data_frames_zhunt/zhunt_cls_df_{virus_name}.csv'
    file_path_rowl = f'/content/row_linkage_matrices_zhunt/zhunt_rowl_{virus_name}.npy'

    clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
    row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
    color_threshold = color_thresholds_zhunt[virus_name]

    dendro = plot_time_length_regression(meta_df=meta_data_all_species,
                            virus_name=virus_name,
                            clustered_dataframe=clustered_dataframe,
                            row_linkage_matrix=row_linkage_matrix,
                            color_threshold = color_threshold,
                            point_size = 20,
                            intervals_column='ZHUNT intervals total length',
                            title=f'{virus}\nZHUNT predictions',
                            figsize=(15, 10), remove_outliers=(True, 3))

# Анализ участков Z-РНК SARS-CoV-2

In [ ]:
sars2_meta = extract_genbank_data(['/content/SARS-CoV-2-10k-new.gbk'])

In [ ]:
sars2_meta = pd.read_csv('sars2_meta_10k.csv')

In [ ]:
# Combine DNABERT prediction files into one dictionary
combined_sars2_dnabert = {}
for file_path in list_files('/content/SARS-CoV-2-parts/DNABERT'):
    with open(file_path, 'r') as f:
        combined_sars2_dnabert.update(json.load(f))

# Update the metadata dataframe with DNABERT information
sars2_meta['ZDNABERT intervals total length'] = sars2_meta['accession_with_version'].apply(
    lambda x: sum_intervals(combined_sars2_dnabert.get(x))
)
sars2_meta['ZDNABERT intervals number'] = sars2_meta['accession_with_version'].apply(
    lambda x: len(combined_sars2_dnabert.get(x))
)

# Combine ZHUNT prediction files into one dictionary
combined_sars2_zhunt = {}
for file_path in list_files('/content/SARS-CoV-2-parts/ZHUNT'):
    with open(file_path, 'r') as f:
        combined_sars2_zhunt.update(json.load(f))

# Update the metadata dataframe with ZHUNT information
sars2_meta['ZHUNT intervals total length'] = sars2_meta['accession_with_version'].apply(
    lambda x: sum_intervals(combined_sars2_zhunt.get(x))
)
sars2_meta['ZHUNT intervals number'] = sars2_meta['accession_with_version'].apply(
    lambda x: len(combined_sars2_zhunt.get(x))
)

In [ ]:
sars2_meta.to_csv('sars2_meta_10k.csv')

In [ ]:
!mkdir SARS-CoV-2

In [ ]:
dnabert_cls_df, dnabert_rowl = create_clustered_dataframe(
    read_json_file('/content/SARS-CoV-2-parts/DNABERT/SARS-CoV-2-part-0_dnabert.json')
)
dnabert_cls_df.to_csv('/content/SARS-CoV-2/dnabert_cls_df_SARS-CoV-2_0.csv')
np.save("/content/SARS-CoV-2/dnabert_rowl_SARS-CoV-2_0.npy", dnabert_rowl)

In [ ]:
dnabert_cls_df, dnabert_rowl = create_clustered_dataframe(
    read_json_file('/content/SARS-CoV-2-parts/DNABERT/SARS-CoV-2-part-0_dnabert.json')
)
dnabert_cls_df.to_csv('/content/SARS-CoV-2/zhunt_cls_df_SARS-CoV-2_0.csv')
np.save("/content/SARS-CoV-2/zhunt_rowl_SARS-CoV-2_0.npy", dnabert_rowl)

In [ ]:
# Calculate color threshold for dnabert predictions
color_thresholds_sars2_dnabert = {}

file_path_rowl = "/content/SARS-CoV-2/dnabert_rowl_SARS-CoV-2_0.npy"
row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
color_threshold = optimal_dendrogram_threshold_and_clusters(row_linkage_matrix, 5)
color_thresholds_sars2_dnabert['SARS-CoV-2'] = color_threshold
save_json(color_thresholds_sars2_dnabert, 'color_thresholds_sars2_dnabert.json')

# Calculate color threshold for zhunt predictions
color_thresholds_sars2_zhunt = {}

file_path_rowl = "/content/SARS-CoV-2/zhunt_rowl_SARS-CoV-2_0.npy"
row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
color_threshold = optimal_dendrogram_threshold_and_clusters(row_linkage_matrix, 5)
color_thresholds_sars2_zhunt['SARS-CoV-2'] = color_threshold
save_json(color_thresholds_sars2_zhunt, 'color_thresholds_sars2_zhunt.json')

In [ ]:
genomic_regions_sars2 = extract_regions_from_genbank_id('NC_045512.2')

In [ ]:
genomic_regions_sars2 = [
    ("5'UTR", 1, 265), ('nsp1', 266, 805), ('nsp2', 806, 2719),
    ('nsp3', 2720, 8554), ('nsp4', 8555, 10054), ('nsp5', 10055, 10972),
    ('nsp6', 10973, 11842), ('nsp7', 11843, 12091), ('nsp8', 12092, 12685),
    ('nsp9', 12686, 13024), ('nsp10', 13025, 13441), ('nsp11', 13442, 13480),
    ('nsp12', 13442, 16236), ('nsp13', 16237, 18039), ('nsp14', 18040, 19620),
    ('nsp15', 19621, 20658), ('nsp16', 20659, 21552), ('S', 21563, 25384),
    ('ORF3a', 25393, 26220), ('E', 26245, 26472), ('M', 26523, 27191),
    ('ORF6', 27202, 27387), ('ORF7a', 27394, 27759), ('ORF7b', 27756, 27887),
    ('ORF8', 27894, 28259), ('N', 28274, 29533), ('ORF10', 29558, 29674),
    ("3'UTR", 29675, 29903)
]

In [ ]:
# SARS-CoV-2
virus = 'SARS-CoV-2'

dnabert_intervals = read_json_file('/content/SARS-CoV-2-parts/DNABERT/SARS-CoV-2-part-0_dnabert.json')
zhunt_intervals = read_json_file('/content/SARS-CoV-2-parts/ZHUNT/SARS-CoV-2-part-0_zhunt.json')
sorted_keys = dnabert_cls_df.columns.to_list()

plot_zna_regions("SARS-CoV-2", sorted_keys, dnabert_intervals, zhunt_intervals, genomic_regions_sars2, colors)

In [ ]:
# Suppress the specific warning from scipy
warnings.filterwarnings("ignore", category=UserWarning, module="scipy.cluster.hierarchy")

# Plot clusters dendrogram and heatmap for SARS-CoV-2 1k sequences (DNABERT)

file_path_df = '/content/SARS-CoV-2/dnabert_cls_df_SARS-CoV-2_0.csv'
file_path_rowl = '/content/SARS-CoV-2/dnabert_rowl_SARS-CoV-2_0.npy'

clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)

title = "SARS-CoV-2 (ZDNABERT)"

color_threshold = color_thresholds_sars2_dnabert['SARS-CoV-2']

plot_heatmap_with_dendrogram(clustered_dataframe, row_linkage_matrix, color_threshold,
                              title=title, figsize=(10, 10), save_figure=True, file_name='SARS-CoV-2')

In [ ]:
# Suppress the specific warning from scipy
warnings.filterwarnings("ignore", category=UserWarning, module="scipy.cluster.hierarchy")

# Plot clusters dendrogram and heatmap for SARS-CoV-2 1k sequences (ZHUNT)

file_path_df = "/content/SARS-CoV-2/zhunt_cls_df_SARS-CoV-2_0.csv"
file_path_rowl = "/content/SARS-CoV-2/zhunt_rowl_SARS-CoV-2_0.npy"

clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)

title = "SARS-CoV-2 (ZHUNT)"

color_threshold = color_thresholds_sars2_zhunt['SARS-CoV-2']

plot_heatmap_with_dendrogram(clustered_dataframe, row_linkage_matrix, color_threshold,
                              title=title, figsize=(10, 10), save_figure=True, file_name='SARS-CoV-2')

In [ ]:
# Add Z-RNA intervals information to the metadata dataframe
sars2_0_ids = pd.read_csv('/content/SARS-CoV-2/dnabert_cls_df_SARS-CoV-2_0.csv', index_col=0).columns.to_list()
meta_sars2_0 = sars2_meta[sars2_meta['accession_with_version'].isin(sars2_0_ids)]
combined_dict_dnabert = read_json_file('/content/SARS-CoV-2-parts/DNABERT/SARS-CoV-2-part-0_dnabert.json')

# Update the metadata dataframe with DNABERT information
meta_sars2_0['ZDNABERT intervals total length'] = meta_sars2_0['accession_with_version'].apply(
    lambda x: sum_intervals(combined_dict_dnabert.get(x))
)
meta_sars2_0['ZDNABERT intervals number'] = meta_sars2_0['accession_with_version'].apply(
    lambda x: len(combined_dict_dnabert.get(x))
)

combined_dict_zhunt = read_json_file('/content/SARS-CoV-2-parts/ZHUNT/SARS-CoV-2-part-0_zhunt.json')

# Update the metadata dataframe with ZHUNT information
meta_sars2_0['ZHUNT intervals total length'] = meta_sars2_0['accession_with_version'].apply(
    lambda x: sum_intervals(combined_dict_zhunt.get(x, None))
)
meta_sars2_0['ZHUNT intervals number'] = meta_sars2_0['accession_with_version'].apply(
    lambda x: len(combined_dict_zhunt.get(x, None))
)

In [ ]:
# Plot intervals total length vs date for ZDNABERT predictions

virus = "SARS-CoV-2"
file_path_df = '/content/SARS-CoV-2/dnabert_cls_df_SARS-CoV-2_0.csv'
file_path_rowl = '/content/SARS-CoV-2/dnabert_rowl_SARS-CoV-2_0.npy'

clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
color_threshold = color_thresholds_sars2_dnabert['SARS-CoV-2']

dendro = plot_time_length_regression(meta_df=meta_sars2_0,
                        virus_name='SARS-CoV-2-10k-new',
                        clustered_dataframe=clustered_dataframe,
                        row_linkage_matrix=row_linkage_matrix,
                        color_threshold = color_threshold,
                        point_size = 10,
                        intervals_column='ZDNABERT intervals total length',
                        title=f'{virus}\nZDNABERT predictions',
                        figsize=(15, 10), remove_outliers=(True, 3))

In [ ]:
# Plot intervals total length vs date for ZHUNT predictions

virus = "SARS-CoV-2"
file_path_df = '/content/SARS-CoV-2/zhunt_cls_df_SARS-CoV-2_0.csv'
file_path_rowl = '/content/SARS-CoV-2/zhunt_rowl_SARS-CoV-2_0.npy'

clustered_dataframe = pd.read_csv(file_path_df, index_col=0)
row_linkage_matrix = np.load(file_path_rowl, allow_pickle=True)
color_threshold = color_thresholds_sars2_zhunt['SARS-CoV-2']

dendro = plot_time_length_regression(meta_df=meta_sars2_0,
                        virus_name='SARS-CoV-2-10k-new',
                        clustered_dataframe=clustered_dataframe,
                        row_linkage_matrix=row_linkage_matrix,
                        color_threshold = color_threshold,
                        point_size = 10,
                        intervals_column='ZHUNT intervals total length',
                        title=f'{virus}\nZHUNT predictions',
                        figsize=(15, 10), remove_outliers=(True, 3))

In [ ]:
# Add WHO lineage information
sars2_who_labels = pd.read_csv('/content/SARS-CoV-2-full-genomes-only.csv')
print('All complete genome strains count: ', sars2_who_labels.shape[0])

sars2_meta = pd.read_csv('/content/sars2_meta_10k.csv')
print('analyzed strains count: ', sars2_meta.shape[0])

sars2_who_labels_dict = dict(zip(sars2_who_labels['Accession'], sars2_who_labels['Pangolin']))
sars2_meta['Pango'] = sars2_meta['accession_with_version'].apply(lambda x: sars2_who_labels_dict.get(x))

sars2_meta['WHO'] = sars2_meta['Pango'].apply(lambda x: pango_to_who(x))

sars2_meta = remove_outliers(sars2_meta, ['GC content'], multiplier=3)
sars2_meta = remove_outliers(sars2_meta, ['ZDNABERT intervals total length'], multiplier=3)
sars2_meta = remove_outliers(sars2_meta, ['ZDNABERT intervals number'], multiplier=3)
print('Outlier strains count: ', 10000 - len(sars2_meta))

In [ ]:
# Plot ZDNABERT intervals total length for WHO linages
draw_boxplot_who(
        df_subset = sars2_meta,
        label = 'WHO',
        values = 'ZDNABERT intervals total length',
        figsize = (15, 6),
        pos = (0.02, 0.02)
        )

In [ ]:
# Plot ZDNABERT intervals number for WHO linages
draw_boxplot_who(
        df_subset = sars2_meta,
        label = 'WHO',
        values = 'ZDNABERT intervals number',
        figsize = (15, 6),
        pos = (0.02, 0.02)
        )

In [ ]:
# Plot ZHUNT intervals total length for WHO linages
draw_boxplot_who(
        df_subset = sars2_meta,
        label = 'WHO',
        values = 'ZHUNT intervals total length',
        figsize = (15, 6),
        pos = (0.02, 0.02)
        )

In [ ]:
# Plot ZHUNT intervals number for WHO linages
draw_boxplot_who(
        df_subset = sars2_meta,
        label = 'WHO',
        values = 'ZHUNT intervals number',
        figsize = (15, 6),
        pos = (0.02, 0.02)
        )

In [ ]:
# Plot GC content for WHO linages
draw_boxplot_who(
        df_subset = sars2_meta,
        label = 'WHO',
        values = 'GC content',
        figsize = (15, 6),
        pos = (0.02, 0.02)
        )

In [ ]:
# Plot ZDNABERT intervals total length by collection date
plot_time_length_linages_boxplot(sars2_meta, 'ZDNABERT intervals total length', figsize = (20, 8))

In [ ]:
# Plot ZHUNT intervals total length by collection date
plot_time_length_linages_boxplot(sars2_meta, 'ZHUNT intervals total length', figsize = (20, 8))

In [ ]:
#Plot ZDNABERT intervals total length by collection date for WHO lineages
plot_time_length_regression_linages(sars2_meta, 'ZDNABERT intervals total length', figsize = (20, 8))

In [ ]:
#Plot ZHUNT intervals total length by collection date for WHO lineages
plot_time_length_regression_linages(sars2_meta, 'ZHUNT intervals total length', figsize = (20, 8))

In [ ]:
#Plot ZDNABERT intervals total length by collection date for Pangolin lineages
plot_time_length_regression_linages(sars2_meta,
                                    'ZDNABERT intervals total length',
                                    category_column='Pango',
                                    figsize = (20, 8), show_legend=False)